In [1]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras import optimizers
import numpy as np
from keras.layers.core import Lambda
from keras import backend as K
from keras.optimizers import SGD
from keras import regularizers

import tensorflow as tf
from keras import backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
K.set_session(sess)

from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten
from keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D
from keras import models
from keras import layers
from keras.models import Model
import matplotlib.pyplot as plt
from keras.models import Sequential

from keras.callbacks import TensorBoard

C:\Users\dell\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#import data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)
 
weight_decay = 0.0005
nb_epoch=200
batch_size=128

In [8]:
from keras.models import load_model
model = load_model('ent_b_cifar10vgg16.h5')
model.load_weights('ent_b_cifar10vgg16_weights.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 64)        256       
__________

In [9]:
tensor =[]
for i in range(0,64):
   
    u=model.get_layer('conv2d_1').get_weights()[0][:,:,:,i].squeeze()
    v=np.mean(u, axis=1)
    #print(v)
    vT=v.T
    D=np.cov(vT)
    try:                 
        invD=np.linalg.inv(D)
        #t=np.mean(v,axis=1)
        a=[]

        for j in range(0,2): #表示filter的大小
            for k in range(j+1,3):
               #if (j!=k) and (j<k):
                tp=v[j]-v[k]
                d=np.sqrt(abs(np.dot(np.dot(tp,invD),tp.T)))
                a.append(d)
            
        print(i,a)
        tensor.append(a)
    except:
        print("不可逆")
print(tensor)

0 [2.3997506203331214, 10.95519264285115, 5.753583268412776]
1 [2.1662894678675215, 2.139904040030092, 1.9406053821597986]
2 [2.44594263604803, 2.519599207360163, 1.6721282729322682]
3 [2.003215567865559, 3.988766083124552, 3.9790453663490584]
4 [1.9073046840639833, 4.063044174134465, 4.590519307518806]
5 [3.4785299627489445, 7.840591113738341, 3.1402471229625686]
6 [0.4373052734166848, 1.9772984978613624, 2.103282116241886]
7 [2.173304245330929, 4.437238658677545, 3.9743823837209287]
8 [0.9199068914598406, 2.192425645439566, 1.9964459364237233]
9 [19.88371023613341, 1.9999958757625331, 3.1753046220920185]
10 [1.244175884998781, 1.8948090968938995, 1.2543564223057526]
11 [1.362592475327363, 2.2849727718613857, 0.9824694817234325]
12 [10.023429527614995, 2.5022329395618796, 10.2647268288484]
13 [2.026191249701093, 10.915487580044577, 9.51404176265908]
14 [0.5427745224707468, 4.793316767088728, 10.816772013907027]
15 [2.350303338135314, 5.870037174139637, 2.0591829386919462]
16 [2.128787

In [10]:
entt=[]
for i in range (0,64):
    
    data=tensor[i]
    data0=np.array(data)
    print('四舍五入，精确到个位\n',np.round(data0,decimals=-2))
    data1=np.round(data0,decimals=-1)

    data1_value_list=set([data1[i] for i in range (data1.shape[0])])
    print(data1_value_list)
    ent=0.0
    for data1_value in data1_value_list:
        p=float(data1[data1==data1_value].shape[0])/data1.shape[0]
        print(p)
        logp=np.log2(p)
        ent-=p*logp
        print(ent)   
    entt.append(ent)
print(entt)

e=np.mean(entt)
print(e)
bb=[k for k in range(len(entt)) if entt[k]<np.mean(entt)]
print(bb)
c=len(bb)
print(c)

四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{0.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{0.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{0.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{0.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{0.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{0.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.918295834054489

In [11]:
from kerassurgeon.operations import delete_layer,insert_layer,delete_channels
model_1 = delete_channels(model, model.layers[0],bb)
model_1.summary()
model_1.save('ent_s_cifar10vgg16_1.h5')
model_1.save_weights('ent_s_cifar10vgg16_1_weights.h5',overwrite=True)

Deleting 29/64 channels from layer: conv2d_1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 35)        980       
_________________________________________________________________
activation_1 (Activation)    multiple                  0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 35)        140       
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        20224     
_________________________________________________________________
activation_2 (Activation)    (N

In [12]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model_1.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])
 
train_history=model_1.fit(x_train,y_train,epochs=200, batch_size=128,
             validation_split=0.1, verbose=1)

model_1.save('ent_s_cifar10vgg16_1.h5')
model_1.save_weights('ent_s_cifar10vgg16_1_weights.h5',overwrite=True)

Train on 45000 samples, validate on 5000 samples
Epoch 1/200
45000/45000 [==============================] - 39s 874us/step - loss: 0.7106 - acc: 0.9629 - val_loss: 1.2364 - val_acc: 0.8440
Epoch 2/200
45000/45000 [==============================] - 28s 618us/step - loss: 0.7299 - acc: 0.9613 - val_loss: 1.1665 - val_acc: 0.8578
Epoch 3/200
45000/45000 [==============================] - 28s 617us/step - loss: 0.7311 - acc: 0.9635 - val_loss: 1.1732 - val_acc: 0.8606
Epoch 4/200
45000/45000 [==============================] - 28s 615us/step - loss: 0.7393 - acc: 0.9624 - val_loss: 1.1441 - val_acc: 0.8718
Epoch 5/200
45000/45000 [==============================] - 28s 616us/step - loss: 0.7410 - acc: 0.9627 - val_loss: 1.0953 - val_acc: 0.8774
Epoch 6/200
45000/45000 [==============================] - 28s 620us/step - loss: 0.7333 - acc: 0.9661 - val_loss: 1.1922 - val_acc: 0.8552
Epoch 7/200
45000/45000 [==============================] - 28s 617us/step - loss: 0.7323 - acc: 0.9662 - val_lo

Epoch 59/200
45000/45000 [==============================] - 28s 622us/step - loss: 0.7199 - acc: 0.9669 - val_loss: 1.0972 - val_acc: 0.8778
Epoch 60/200
45000/45000 [==============================] - 28s 611us/step - loss: 0.7128 - acc: 0.9692 - val_loss: 1.1943 - val_acc: 0.8626
Epoch 61/200
45000/45000 [==============================] - 30s 676us/step - loss: 0.7113 - acc: 0.9686 - val_loss: 1.1697 - val_acc: 0.8662
Epoch 62/200
45000/45000 [==============================] - 30s 677us/step - loss: 0.7092 - acc: 0.9702 - val_loss: 1.0989 - val_acc: 0.8790
Epoch 63/200
45000/45000 [==============================] - 30s 677us/step - loss: 0.7091 - acc: 0.9694 - val_loss: 1.1276 - val_acc: 0.8780
Epoch 64/200
45000/45000 [==============================] - 30s 677us/step - loss: 0.7106 - acc: 0.9682 - val_loss: 1.2046 - val_acc: 0.8572
Epoch 65/200
45000/45000 [==============================] - 31s 679us/step - loss: 0.7043 - acc: 0.9709 - val_loss: 1.1151 - val_acc: 0.8768
Epoch 66/200


Epoch 174/200
45000/45000 [==============================] - 31s 678us/step - loss: 0.7127 - acc: 0.9670 - val_loss: 1.1972 - val_acc: 0.8544
Epoch 175/200
45000/45000 [==============================] - 30s 676us/step - loss: 0.7071 - acc: 0.9688 - val_loss: 1.1047 - val_acc: 0.8756
Epoch 176/200
45000/45000 [==============================] - 31s 681us/step - loss: 0.7016 - acc: 0.9697 - val_loss: 1.0925 - val_acc: 0.8782
Epoch 177/200
45000/45000 [==============================] - 31s 679us/step - loss: 0.7177 - acc: 0.9648 - val_loss: 1.0948 - val_acc: 0.8830
Epoch 178/200
45000/45000 [==============================] - 30s 676us/step - loss: 0.7089 - acc: 0.9687 - val_loss: 1.0798 - val_acc: 0.8854
Epoch 179/200
45000/45000 [==============================] - 30s 675us/step - loss: 0.7045 - acc: 0.9703 - val_loss: 1.0968 - val_acc: 0.8794
Epoch 180/200
45000/45000 [==============================] - 31s 680us/step - loss: 0.7008 - acc: 0.9708 - val_loss: 1.1135 - val_acc: 0.8732
Epoch 

In [13]:
import functools
import keras
top1_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=1) #top-1精度

top1_acc.__name__ = 'top1_acc'

model_1.compile(loss='categorical_crossentropy',
             optimizer='adam',metrics=['accuracy','top_k_categorical_accuracy',top1_acc])

model_1.evaluate(x_test,y_test,verbose=0)


[1.164666438293457, 0.8612, 0.991, 0.8612]

In [14]:

score=model_1.evaluate(x_test,y_test,verbose=0)
print("测试损失为：{:.4f}".format(score[0]))
print("测试准确率为：{:.4f}".format(score[1]))

测试损失为：1.1647
测试准确率为：0.8612


In [15]:
tensor =[]
for i in range(0,64):
   
    u=model_1.get_layer('conv2d_2').get_weights()[0][:,:,:,i].squeeze()
    v=np.mean(u, axis=1)
    #print(v)
    vT=v.T
    D=np.cov(vT)
    try:                 
        invD=np.linalg.inv(D)
        #t=np.mean(v,axis=1)
        a=[]

        for j in range(0,2): #表示filter的大小
            for k in range(j+1,3):
               #if (j!=k) and (j<k):
                tp=v[j]-v[k]
                d=np.sqrt(abs(np.dot(np.dot(tp,invD),tp.T)))
                a.append(d)
            
        print(i,a)
        tensor.append(a)
    except:
        print("不可逆")
print(tensor)

0 [11.327524187663734, 6.081917410724117, 20.450954663067918]
1 [4.205793873512565, 1.2905430700792293, 12.790972455764877]
2 [11.845922789144769, 9.747585878074604, 3.735286298194146]
3 [16.55975480271301, 31.7515282000318, 19.601841620126226]
4 [34.497784529606704, 7.7522852819994865, 43.35654076463199]
5 [16.21115465133644, 30.902437320124424, 2.913068095455334]
6 [19.002806569163287, 7.521142690817723, 4.978289389904306]
7 [115.57515150914709, 97.36469877353261, 31.467064487507212]
8 [4.290817096408082, 19.584384085149562, 18.297232906968457]
9 [10.29803279682863, 19.46791836598769, 5.351047730573927]
10 [46.04077339974408, 22.546159316538986, 9.444115940782327]
11 [13.066435855210411, 14.220152008187119, 8.384160369844395]
12 [9.154623818098713, 9.0516436301411, 6.042430836483347]
13 [5.770530846537365, 13.93412922182389, 11.785137577313938]
14 [5.38781966037738, 12.236544430123146, 11.915588121134515]
15 [4.363850418985344, 24.13968642302146, 10.055676605018217]
16 [16.7122280198

In [16]:
entt=[]
for i in range (0,64):
    
    data=tensor[i]
    data0=np.array(data)
    print('四舍五入，精确到个位\n',np.round(data0,decimals=-2))
    data1=np.round(data0,decimals=-1)

    data1_value_list=set([data1[i] for i in range (data1.shape[0])])
    print(data1_value_list)
    ent=0.0
    for data1_value in data1_value_list:
        p=float(data1[data1==data1_value].shape[0])/data1.shape[0]
        print(p)
        logp=np.log2(p)
        ent-=p*logp
        print(ent)   
    entt.append(ent)
print(entt)

e=np.mean(entt)
print(e)
bb=[k for k in range(len(entt)) if entt[k]<np.mean(entt)]
print(bb)
c=len(bb)
print(c)

四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{20.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 10.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [100. 100.   0.]
{120.0, 100.0, 30.0}
0.3333333333333333
0.5283208335737187


In [17]:
from kerassurgeon.operations import delete_layer,insert_layer,delete_channels
model_2 = delete_channels(model_1, model_1.layers[5],bb)
model_2.summary()
model_2.save('ent_b_cifar10vgg16_12.h5')
model_2.save_weights('ent_b_cifar10vgg16_12_weights.h5',overwrite=True)

Deleting 46/64 channels from layer: conv2d_2
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 35)        980       
_________________________________________________________________
activation_1 (Activation)    multiple                  0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 35)        140       
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 18)        5688      
_________________________________________________________________
activation_2 (Activation)    mu

In [18]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model_2.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])
 
train_history=model_2.fit(x_train,y_train,epochs=200, batch_size=128,
             validation_split=0.1, verbose=1)

model_2.save('ent_s_cifar10vgg16_2.h5')
model_2.save_weights('ent_s_cifar10vgg16_2_weights.h5',overwrite=True)

Train on 45000 samples, validate on 5000 samples
Epoch 1/200
45000/45000 [==============================] - 37s 830us/step - loss: 0.8187 - acc: 0.9314 - val_loss: 1.0950 - val_acc: 0.8668
Epoch 2/200
45000/45000 [==============================] - 29s 654us/step - loss: 0.7826 - acc: 0.9459 - val_loss: 1.1796 - val_acc: 0.8492
Epoch 3/200
45000/45000 [==============================] - 30s 656us/step - loss: 0.7763 - acc: 0.9528 - val_loss: 1.2389 - val_acc: 0.8472
Epoch 4/200
45000/45000 [==============================] - 30s 656us/step - loss: 0.7666 - acc: 0.9581 - val_loss: 1.1768 - val_acc: 0.8618
Epoch 5/200
45000/45000 [==============================] - 29s 643us/step - loss: 0.7702 - acc: 0.9582 - val_loss: 1.1309 - val_acc: 0.8684
Epoch 6/200
45000/45000 [==============================] - 30s 660us/step - loss: 0.7685 - acc: 0.9598 - val_loss: 1.1735 - val_acc: 0.8670
Epoch 7/200
45000/45000 [==============================] - 30s 656us/step - loss: 0.7658 - acc: 0.9612 - val_lo

Epoch 59/200
45000/45000 [==============================] - 30s 656us/step - loss: 0.7672 - acc: 0.9613 - val_loss: 1.2303 - val_acc: 0.8568
Epoch 60/200
45000/45000 [==============================] - 30s 659us/step - loss: 0.7581 - acc: 0.9640 - val_loss: 1.2306 - val_acc: 0.8500
Epoch 61/200
45000/45000 [==============================] - 30s 658us/step - loss: 0.7610 - acc: 0.9637 - val_loss: 1.1241 - val_acc: 0.8782
Epoch 62/200
45000/45000 [==============================] - 30s 656us/step - loss: 0.7555 - acc: 0.9647 - val_loss: 1.1905 - val_acc: 0.8620
Epoch 63/200
45000/45000 [==============================] - 30s 656us/step - loss: 0.7512 - acc: 0.9666 - val_loss: 1.1574 - val_acc: 0.8724
Epoch 64/200
45000/45000 [==============================] - 30s 660us/step - loss: 0.7450 - acc: 0.9672 - val_loss: 1.1801 - val_acc: 0.8682
Epoch 65/200
45000/45000 [==============================] - 29s 655us/step - loss: 0.7484 - acc: 0.9653 - val_loss: 1.1350 - val_acc: 0.8764
Epoch 66/200


Epoch 117/200
45000/45000 [==============================] - 30s 657us/step - loss: 0.7358 - acc: 0.9678 - val_loss: 1.2117 - val_acc: 0.8576
Epoch 118/200
45000/45000 [==============================] - 30s 659us/step - loss: 0.7375 - acc: 0.9671 - val_loss: 1.2167 - val_acc: 0.8598
Epoch 119/200
45000/45000 [==============================] - 30s 656us/step - loss: 0.7374 - acc: 0.9673 - val_loss: 1.2612 - val_acc: 0.8460
Epoch 120/200
45000/45000 [==============================] - 30s 656us/step - loss: 0.7398 - acc: 0.9666 - val_loss: 1.2137 - val_acc: 0.8606
Epoch 121/200
45000/45000 [==============================] - 30s 660us/step - loss: 0.7488 - acc: 0.9641 - val_loss: 1.1720 - val_acc: 0.8632
Epoch 122/200
45000/45000 [==============================] - 30s 656us/step - loss: 0.7364 - acc: 0.9676 - val_loss: 1.2088 - val_acc: 0.8550
Epoch 123/200
45000/45000 [==============================] - 30s 658us/step - loss: 0.7298 - acc: 0.9697 - val_loss: 1.2289 - val_acc: 0.8596
Epoch 

Epoch 175/200
45000/45000 [==============================] - 29s 652us/step - loss: 0.7221 - acc: 0.9680 - val_loss: 1.1836 - val_acc: 0.8640
Epoch 176/200
45000/45000 [==============================] - 30s 657us/step - loss: 0.7271 - acc: 0.9660 - val_loss: 1.1586 - val_acc: 0.8688
Epoch 177/200
45000/45000 [==============================] - 30s 658us/step - loss: 0.7259 - acc: 0.9673 - val_loss: 1.2127 - val_acc: 0.8542
Epoch 178/200
45000/45000 [==============================] - 30s 657us/step - loss: 0.7234 - acc: 0.9676 - val_loss: 1.1387 - val_acc: 0.8764
Epoch 179/200
45000/45000 [==============================] - 30s 657us/step - loss: 0.7323 - acc: 0.9662 - val_loss: 1.1726 - val_acc: 0.8662
Epoch 180/200
45000/45000 [==============================] - 29s 654us/step - loss: 0.7213 - acc: 0.9696 - val_loss: 1.1544 - val_acc: 0.8710
Epoch 181/200
45000/45000 [==============================] - 30s 658us/step - loss: 0.7256 - acc: 0.9676 - val_loss: 1.2664 - val_acc: 0.8456
Epoch 

In [19]:
import functools
import keras
top1_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=1) #top-1精度

top1_acc.__name__ = 'top1_acc'

model_2.compile(loss='categorical_crossentropy',
             optimizer='adam',metrics=['accuracy','top_k_categorical_accuracy',top1_acc])

model_2.evaluate(x_test,y_test,verbose=0)


[1.1680644845962525, 0.8632, 0.9898, 0.8632]

In [20]:
score=model_2.evaluate(x_test,y_test,verbose=0)
print("测试损失为：{:.4f}".format(score[0]))
print("测试准确率为：{:.4f}".format(score[1]))

测试损失为：1.1681
测试准确率为：0.8632


In [21]:
tensor =[]
for i in range(0,128):
   
    u=model_2.get_layer('conv2d_3').get_weights()[0][:,:,:,i].squeeze()
    v=np.mean(u, axis=1)
    #print(v)
    vT=v.T
    D=np.cov(vT)
    try:                 
        invD=np.linalg.inv(D)
        #t=np.mean(v,axis=1)
        a=[]

        for j in range(0,2): #表示filter的大小
            for k in range(j+1,3):
               #if (j!=k) and (j<k):
                tp=v[j]-v[k]
                d=np.sqrt(abs(np.dot(np.dot(tp,invD),tp.T)))
                a.append(d)
            
        print(i,a)
        tensor.append(a)
    except:
        print("不可逆")
print(tensor)

0 [30.723161646426124, 5.7871957258154385, 24.05746605471226]
1 [8.492455001871303, 21.98337963158179, 23.752117263381137]
2 [9.429452155209638, 18.27878516119149, 26.763895008944505]
3 [11.918847345795793, 8.271119481925252, 2.1012250210958308]
4 [4.648563697448278, 7.754340340789058, 9.284313685247774]
5 [9.44470978384403, 1.7992945593409535, 18.10874009685676]
6 [4.828555140423489, 12.690190936576759, 12.07073878057122]
7 [37.35619174441511, 64.66205890533831, 22.49514921669225]
8 [7.343121992814015, 5.100662198739136, 13.158823137707882]
9 [1.7360641913961445, 11.15557280122666, 2.102671731111035]
10 [12.486364833873894, 22.092859814096496, 12.626806177167825]
11 [8.05063729340265, 8.928607331339995, 7.293499314838811]
12 [23.27592636788891, 6.754941209047356, 2.4461208140978608]
13 [13.693725459165185, 2.4536543839840834, 18.86630321507565]
14 [7.96396113094116, 7.306978537570146, 5.517497028666337]
15 [7.658284738266787, 17.37142436195646, 8.150720351510973]
16 [5.610174648061911

In [22]:
entt=[]
for i in range (0,128):
    
    data=tensor[i]
    data0=np.array(data)
    print('四舍五入，精确到个位\n',np.round(data0,decimals=-2))
    data1=np.round(data0,decimals=-1)

    data1_value_list=set([data1[i] for i in range (data1.shape[0])])
    print(data1_value_list)
    ent=0.0
    for data1_value in data1_value_list:
        p=float(data1[data1==data1_value].shape[0])/data1.shape[0]
        print(p)
        logp=np.log2(p)
        ent-=p*logp
        print(ent)   
    entt.append(ent)
print(entt)

e=np.mean(entt)
print(e)
bb=[k for k in range(len(entt)) if entt[k]<np.mean(entt)]
print(bb)
c=len(bb)
print(c)

四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [  0. 100.   0.]
{40.0, 60.0, 20.0}
0.3333333333333333
0.5283208335737187
0.333

 [0. 0. 0.]
{40.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [  0. 100.   0.]
{40.0, 50.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0

In [23]:
from kerassurgeon.operations import delete_layer,insert_layer,delete_channels
model_3 = delete_channels(model_2, model_2.layers[9],bb)
model_3.summary()
model_3.save('ent_s_cifar10vgg16_3.h5')
model_3.save_weights('ent_s_cifar10vgg16_3_weights.h5',overwrite=True)

Deleting 30/128 channels from layer: conv2d_3
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 35)        980       
_________________________________________________________________
activation_1 (Activation)    multiple                  0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 35)        140       
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 18)        5688      
_________________________________________________________________
activation_2 (Activation)    m

In [24]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model_3.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])
 
train_history=model_3.fit(x_train,y_train,epochs=200, batch_size=128,
             validation_split=0.1, verbose=1)

model_3.save('ent_s_cifar10vgg16_3.h5')
model_3.save_weights('ent_s_cifar10vgg16_3_weights.h5',overwrite=True)

Train on 45000 samples, validate on 5000 samples
Epoch 1/200
45000/45000 [==============================] - 38s 840us/step - loss: 0.7469 - acc: 0.9578 - val_loss: 1.1232 - val_acc: 0.8700
Epoch 2/200
45000/45000 [==============================] - 29s 641us/step - loss: 0.7665 - acc: 0.9569 - val_loss: 1.1909 - val_acc: 0.8646
Epoch 3/200
45000/45000 [==============================] - 29s 640us/step - loss: 0.7616 - acc: 0.9602 - val_loss: 1.1737 - val_acc: 0.8630
Epoch 4/200
45000/45000 [==============================] - 29s 637us/step - loss: 0.7560 - acc: 0.9633 - val_loss: 1.1528 - val_acc: 0.8682
Epoch 5/200
45000/45000 [==============================] - 29s 642us/step - loss: 0.7647 - acc: 0.9624 - val_loss: 1.2281 - val_acc: 0.8642
Epoch 6/200
45000/45000 [==============================] - 25s 557us/step - loss: 0.7747 - acc: 0.9603 - val_loss: 1.2564 - val_acc: 0.8480
Epoch 7/200
45000/45000 [==============================] - 26s 586us/step - loss: 0.7673 - acc: 0.9636 - val_lo

Epoch 59/200
45000/45000 [==============================] - 29s 641us/step - loss: 0.7653 - acc: 0.9649 - val_loss: 1.2156 - val_acc: 0.8648
Epoch 60/200
45000/45000 [==============================] - 29s 634us/step - loss: 0.7681 - acc: 0.9638 - val_loss: 1.2181 - val_acc: 0.8608
Epoch 61/200
45000/45000 [==============================] - 29s 642us/step - loss: 0.7683 - acc: 0.9645 - val_loss: 1.3071 - val_acc: 0.8412
Epoch 62/200
45000/45000 [==============================] - 29s 640us/step - loss: 0.7639 - acc: 0.9654 - val_loss: 1.2106 - val_acc: 0.8652
Epoch 63/200
45000/45000 [==============================] - 29s 641us/step - loss: 0.7569 - acc: 0.9658 - val_loss: 1.2049 - val_acc: 0.8582
Epoch 64/200
45000/45000 [==============================] - 29s 641us/step - loss: 0.7527 - acc: 0.9667 - val_loss: 1.2253 - val_acc: 0.8608
Epoch 65/200
45000/45000 [==============================] - 29s 639us/step - loss: 0.7531 - acc: 0.9654 - val_loss: 1.1988 - val_acc: 0.8636
Epoch 66/200


Epoch 117/200
45000/45000 [==============================] - 29s 643us/step - loss: 0.7426 - acc: 0.9683 - val_loss: 1.2089 - val_acc: 0.8616
Epoch 118/200
45000/45000 [==============================] - 29s 640us/step - loss: 0.7454 - acc: 0.9678 - val_loss: 1.2129 - val_acc: 0.8678
Epoch 119/200
45000/45000 [==============================] - 29s 642us/step - loss: 0.7614 - acc: 0.9620 - val_loss: 1.2076 - val_acc: 0.8632
Epoch 120/200
45000/45000 [==============================] - 29s 643us/step - loss: 0.7494 - acc: 0.9670 - val_loss: 1.1821 - val_acc: 0.8724
Epoch 121/200
45000/45000 [==============================] - 29s 644us/step - loss: 0.7555 - acc: 0.9644 - val_loss: 1.2175 - val_acc: 0.8628
Epoch 122/200
45000/45000 [==============================] - 29s 641us/step - loss: 0.7527 - acc: 0.9670 - val_loss: 1.1843 - val_acc: 0.8704
Epoch 123/200
45000/45000 [==============================] - 29s 642us/step - loss: 0.7525 - acc: 0.9661 - val_loss: 1.2215 - val_acc: 0.8582
Epoch 

Epoch 175/200
45000/45000 [==============================] - 29s 643us/step - loss: 0.7375 - acc: 0.9679 - val_loss: 1.1820 - val_acc: 0.8670
Epoch 176/200
45000/45000 [==============================] - 29s 641us/step - loss: 0.7383 - acc: 0.9683 - val_loss: 1.1689 - val_acc: 0.8700
Epoch 177/200
45000/45000 [==============================] - 29s 640us/step - loss: 0.7430 - acc: 0.9656 - val_loss: 1.1827 - val_acc: 0.8700
Epoch 178/200
45000/45000 [==============================] - 29s 638us/step - loss: 0.7405 - acc: 0.9661 - val_loss: 1.1533 - val_acc: 0.8716
Epoch 179/200
45000/45000 [==============================] - 29s 642us/step - loss: 0.7348 - acc: 0.9690 - val_loss: 1.1691 - val_acc: 0.8692
Epoch 180/200
45000/45000 [==============================] - 29s 645us/step - loss: 0.7392 - acc: 0.9664 - val_loss: 1.2073 - val_acc: 0.8598
Epoch 181/200
45000/45000 [==============================] - 29s 641us/step - loss: 0.7404 - acc: 0.9673 - val_loss: 1.1755 - val_acc: 0.8650
Epoch 

In [25]:
import functools
import keras
top1_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=1) #top-1精度

top1_acc.__name__ = 'top1_acc'

model_3.compile(loss='categorical_crossentropy',
             optimizer='adam',metrics=['accuracy','top_k_categorical_accuracy',top1_acc])

model_3.evaluate(x_test,y_test,verbose=0)


[1.2114363609313965, 0.858, 0.9899, 0.858]

In [26]:
score=model_3.evaluate(x_test,y_test,verbose=0)
print("测试损失为：{:.4f}".format(score[0]))
print("测试准确率为：{:.4f}".format(score[1]))

测试损失为：1.2114
测试准确率为：0.8580


In [27]:
tensor =[]
for i in range(0,128):
   
    u=model_3.get_layer('conv2d_4').get_weights()[0][:,:,:,i].squeeze()
    v=np.mean(u, axis=1)
    #print(v)
    vT=v.T
    D=np.cov(vT)
    try:                 
        invD=np.linalg.inv(D)
        #t=np.mean(v,axis=1)
        a=[]

        for j in range(0,2): #表示filter的大小
            for k in range(j+1,3):
               #if (j!=k) and (j<k):
                tp=v[j]-v[k]
                d=np.sqrt(abs(np.dot(np.dot(tp,invD),tp.T)))
                a.append(d)
            
        print(i,a)
        tensor.append(a)
    except:
        print("不可逆")
print(tensor)

0 [21.583094373384178, 90.69317756405407, 21.259245073133695]
1 [17.61050146065869, 11.846045601242583, 14.383010058934385]
2 [10.248063340636413, 15.813388709021732, 3.892266842444198]
3 [10.320342093460658, 11.783221780517824, 1.0167518819731043]
4 [27.46969573353393, 15.34367289785992, 16.07374627310056]
5 [10.413594560920343, 23.46632516984559, 9.479956161542525]
6 [11.90662939112988, 13.027906542378677, 24.562680522847426]
7 [10.877448539390782, 14.501481638221465, 8.668623140997914]
8 [26.720043771216808, 56.11419267827064, 56.66440422179119]
9 [19.49023514566134, 25.00963220050558, 14.346203695955243]
10 [20.62725771482337, 29.871224934990888, 18.02959016667369]
11 [36.6841002811057, 8.901909905073087, 19.033066536035037]
12 [30.939922168586943, 11.762090630861758, 42.481329553060164]
13 [16.03311687701403, 18.726057457190894, 13.034572991801967]
14 [12.549170088209081, 29.978939281305554, 16.95131470468095]
15 [21.796056150963686, 17.362717053544507, 37.1151874045906]
16 [21.24

In [28]:
entt=[]
for i in range (0,128):
    
    data=tensor[i]
    data0=np.array(data)
    print('四舍五入，精确到个位\n',np.round(data0,decimals=-2))
    data1=np.round(data0,decimals=-1)

    data1_value_list=set([data1[i] for i in range (data1.shape[0])])
    print(data1_value_list)
    ent=0.0
    for data1_value in data1_value_list:
        p=float(data1[data1==data1_value].shape[0])/data1.shape[0]
        print(p)
        logp=np.log2(p)
        ent-=p*logp
        print(ent)   
    entt.append(ent)
print(entt)

e=np.mean(entt)
print(e)
bb=[k for k in range(len(entt)) if entt[k]<np.mean(entt)]
print(bb)
c=len(bb)
print(c)

四舍五入，精确到个位
 [  0. 100.   0.]
{90.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{20.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [  0. 100. 100.]
{60.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到

0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{20.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [  0.   0. 100.]
{40.0, 80.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 30.0}
0.3333333333333333
0.5283208335737187
0

In [29]:
from kerassurgeon.operations import delete_layer,insert_layer,delete_channels
model_4 = delete_channels(model_3, model_3.layers[13],bb)
model_4.summary()
model_4.save('ent_s_cifar10vgg16_4.h5')
model_4.save_weights('ent_s_cifar10vgg16_4_weights.h5',overwrite=True)

Deleting 95/128 channels from layer: conv2d_4
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 35)        980       
_________________________________________________________________
activation_1 (Activation)    multiple                  0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 35)        140       
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 18)        5688      
_________________________________________________________________
activation_2 (Activation)    m

In [30]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model_4.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])
 
train_history=model_4.fit(x_train,y_train,epochs=200, batch_size=128,
             validation_split=0.1, verbose=1)

model_4.save('ent_s_cifar10vgg16_4.h5')
model_4.save_weights('ent_s_cifar10vgg16_4_weights.h5',overwrite=True)

Train on 45000 samples, validate on 5000 samples
Epoch 1/200
45000/45000 [==============================] - 38s 851us/step - loss: 0.9353 - acc: 0.8976 - val_loss: 1.1997 - val_acc: 0.8368
Epoch 2/200
45000/45000 [==============================] - 29s 635us/step - loss: 0.8427 - acc: 0.9330 - val_loss: 1.2073 - val_acc: 0.8454
Epoch 3/200
45000/45000 [==============================] - 29s 641us/step - loss: 0.8403 - acc: 0.9395 - val_loss: 1.3141 - val_acc: 0.8312
Epoch 4/200
45000/45000 [==============================] - 29s 641us/step - loss: 0.8436 - acc: 0.9413 - val_loss: 1.3236 - val_acc: 0.8266
Epoch 5/200
45000/45000 [==============================] - 29s 636us/step - loss: 0.8369 - acc: 0.9476 - val_loss: 1.2808 - val_acc: 0.8378
Epoch 6/200
45000/45000 [==============================] - 29s 641us/step - loss: 0.8279 - acc: 0.9512 - val_loss: 1.2469 - val_acc: 0.8552
Epoch 7/200
45000/45000 [==============================] - 29s 641us/step - loss: 0.8403 - acc: 0.9491 - val_lo

Epoch 59/200
45000/45000 [==============================] - 29s 638us/step - loss: 0.8185 - acc: 0.9588 - val_loss: 1.2747 - val_acc: 0.8542
Epoch 60/200
45000/45000 [==============================] - 29s 638us/step - loss: 0.8066 - acc: 0.9625 - val_loss: 1.3223 - val_acc: 0.8456
Epoch 61/200
45000/45000 [==============================] - 29s 642us/step - loss: 0.8062 - acc: 0.9617 - val_loss: 1.2980 - val_acc: 0.8470
Epoch 62/200
45000/45000 [==============================] - 29s 640us/step - loss: 0.8087 - acc: 0.9606 - val_loss: 1.2871 - val_acc: 0.8518
Epoch 63/200
45000/45000 [==============================] - 29s 637us/step - loss: 0.8094 - acc: 0.9594 - val_loss: 1.3069 - val_acc: 0.8508
Epoch 64/200
45000/45000 [==============================] - 29s 637us/step - loss: 0.7982 - acc: 0.9632 - val_loss: 1.3080 - val_acc: 0.8518
Epoch 65/200
45000/45000 [==============================] - 29s 637us/step - loss: 0.8002 - acc: 0.9625 - val_loss: 1.2596 - val_acc: 0.8550
Epoch 66/200


Epoch 117/200
45000/45000 [==============================] - 29s 638us/step - loss: 0.8008 - acc: 0.9592 - val_loss: 1.2751 - val_acc: 0.8550
Epoch 118/200
45000/45000 [==============================] - 29s 639us/step - loss: 0.7891 - acc: 0.9645 - val_loss: 1.3745 - val_acc: 0.8350
Epoch 119/200
45000/45000 [==============================] - 29s 639us/step - loss: 0.7997 - acc: 0.9608 - val_loss: 1.3285 - val_acc: 0.8494
Epoch 120/200
45000/45000 [==============================] - 29s 640us/step - loss: 0.8007 - acc: 0.9606 - val_loss: 1.2435 - val_acc: 0.8600
Epoch 121/200
45000/45000 [==============================] - 29s 640us/step - loss: 0.7906 - acc: 0.9638 - val_loss: 1.2214 - val_acc: 0.8598
Epoch 122/200
45000/45000 [==============================] - 29s 641us/step - loss: 0.7892 - acc: 0.9634 - val_loss: 1.2578 - val_acc: 0.8600
Epoch 123/200
45000/45000 [==============================] - 29s 640us/step - loss: 0.7902 - acc: 0.9632 - val_loss: 1.3644 - val_acc: 0.8360
Epoch 

Epoch 175/200
45000/45000 [==============================] - 29s 638us/step - loss: 0.7851 - acc: 0.9645 - val_loss: 1.2922 - val_acc: 0.8546
Epoch 176/200
45000/45000 [==============================] - 29s 634us/step - loss: 0.7828 - acc: 0.9648 - val_loss: 1.2666 - val_acc: 0.8552
Epoch 177/200
45000/45000 [==============================] - 29s 639us/step - loss: 0.7735 - acc: 0.9667 - val_loss: 1.3235 - val_acc: 0.8498
Epoch 178/200
45000/45000 [==============================] - 29s 637us/step - loss: 0.7842 - acc: 0.9635 - val_loss: 1.2707 - val_acc: 0.8464
Epoch 179/200
45000/45000 [==============================] - 28s 632us/step - loss: 0.7909 - acc: 0.9613 - val_loss: 1.3467 - val_acc: 0.8310
Epoch 180/200
45000/45000 [==============================] - 29s 640us/step - loss: 0.7950 - acc: 0.9609 - val_loss: 1.3993 - val_acc: 0.8342
Epoch 181/200
45000/45000 [==============================] - 29s 636us/step - loss: 0.7891 - acc: 0.9627 - val_loss: 1.3630 - val_acc: 0.8326
Epoch 

In [31]:
import functools
import keras
top1_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=1) #top-1精度

top1_acc.__name__ = 'top1_acc'

model_4.compile(loss='categorical_crossentropy',
             optimizer='adam',metrics=['accuracy','top_k_categorical_accuracy',top1_acc])

model_4.evaluate(x_test,y_test,verbose=0)


[1.369804288673401, 0.8244, 0.9834, 0.8244]

In [32]:
score=model_4.evaluate(x_test,y_test,verbose=0)
print("测试损失为：{:.4f}".format(score[0]))
print("测试准确率为：{:.4f}".format(score[1]))

测试损失为：1.3698
测试准确率为：0.8244


In [33]:
tensor =[]
for i in range(0,256):
   
    u=model_4.get_layer('conv2d_5').get_weights()[0][:,:,:,i].squeeze()
    v=np.mean(u, axis=1)
    #print(v)
    vT=v.T
    D=np.cov(vT)
    try:                 
        invD=np.linalg.inv(D)
        #t=np.mean(v,axis=1)
        a=[]

        for j in range(0,2): #表示filter的大小
            for k in range(j+1,3):
               #if (j!=k) and (j<k):
                tp=v[j]-v[k]
                d=np.sqrt(abs(np.dot(np.dot(tp,invD),tp.T)))
                a.append(d)
            
        print(i,a)
        tensor.append(a)
    except:
        print("不可逆")
print(tensor)

0 [6.128461463013433, 10.25737312346276, 3.5162319422372432]
1 [12.52654392982868, 8.14010366941439, 9.195082876341203]
2 [16.546784743800412, 41.49065237593877, 7.70787878156861]
3 [13.15464809206565, 42.885516331114964, 38.84514695814275]
4 [9.281351416447961, 38.06164939443559, 2.6061642932292703]
5 [13.171465630918037, 7.630710636482933, 11.761138769956258]
6 [17.120865406251735, 22.46132382267971, 9.261377631590141]
7 [3.409867352005994, 7.545445806416238, 27.56607810083654]
8 [5.514030636754535, 12.66009035596498, 25.723915295211853]
9 [17.67246120570041, 19.027460732645675, 29.577077195137193]
10 [10.564753275137397, 30.290045910432415, 29.453415538862505]
11 [11.698276427056705, 8.295639908811033, 8.673439613304918]
12 [20.887611965059705, 4.591717017163672, 8.923743110864612]
13 [7.2756508021589, 13.786981303469561, 13.928486154455932]
14 [12.76121657790221, 13.655288374365512, 11.320589640024103]
15 [7.977168584171974, 6.762760705734996, 8.43626418776795]
16 [18.1122001175574

137 [6.422612669963609, 3.054619517630871, 3.850123595136445]
138 [3.883028573177896, 11.250740609692604, 15.792595529875749]
139 [6.022482621914885, 6.0896895354236165, 5.743047066015601]
140 [2.0237908956861825, 19.57238868162312, 11.495255694107042]
141 [9.69684575795944, 13.881947082010187, 16.712609020667898]
142 [3.84722324781408, 3.4198539374208434, 5.813834006523382]
143 [18.579820149745125, 14.475236464497739, 37.72524570261205]
144 [15.283482753157756, 22.72662683964995, 4.363749288593323]
145 [29.372232509677215, 55.494140728110814, 26.06201566616627]
146 [35.78213529199574, 38.45996102428792, 6.107528563599345]
147 [9.330171140223493, 13.492034186732988, 6.322709429474957]
148 [24.435613149596215, 32.438729823324756, 11.995008041256702]
149 [39.52445328374765, 25.109351983620634, 35.85293793923326]
150 [13.348981059761165, 23.412057479529746, 22.05345046019596]
151 [6.496029154444323, 9.358322797674791, 17.6324951521914]
152 [8.274667621402683, 14.871266914403588, 4.6894425

In [34]:
entt=[]
for i in range (0,256):
    
    data=tensor[i]
    data0=np.array(data)
    print('四舍五入，精确到个位\n',np.round(data0,decimals=-2))
    data1=np.round(data0,decimals=-1)

    data1_value_list=set([data1[i] for i in range (data1.shape[0])])
    print(data1_value_list)
    ent=0.0
    for data1_value in data1_value_list:
        p=float(data1[data1==data1_value].shape[0])/data1.shape[0]
        print(p)
        logp=np.log2(p)
        ent-=p*logp
        print(ent)   
    entt.append(ent)
print(entt)

e=np.mean(entt)
print(e)
bb=[k for k in range(len(entt)) if entt[k]<np.mean(entt)]
print(bb)
c=len(bb)
print(c)

四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 10.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 

 [0. 0. 0.]
{40.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.33333333333333

 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{20.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 30.0}
0.6666666666666666
0.38997500048077083
0.33

In [35]:
from kerassurgeon.operations import delete_layer,insert_layer,delete_channels
model_5 = delete_channels(model_4, model_4.layers[17],bb)
model_5.summary()
model_5.save('ent_s_cifar10vgg16_5.h5')
model_5.save_weights('ent_s_cifar10vgg16_5_weights.h5',overwrite=True)

Deleting 39/256 channels from layer: conv2d_5
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 35)        980       
_________________________________________________________________
activation_1 (Activation)    multiple                  0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 35)        140       
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 18)        5688      
_________________________________________________________________
activation_2 (Activation)    m

In [36]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model_5.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])
 
train_history=model_5.fit(x_train,y_train,epochs=200, batch_size=128,
             validation_split=0.1, verbose=1)

model_5.save('ent_s_cifar10vgg16_5.h5')
model_5.save_weights('ent_s_cifar10vgg16_5_weights.h5',overwrite=True)

Train on 45000 samples, validate on 5000 samples
Epoch 1/200
45000/45000 [==============================] - 39s 863us/step - loss: 0.7936 - acc: 0.9559 - val_loss: 1.2479 - val_acc: 0.8524
Epoch 2/200
45000/45000 [==============================] - 28s 622us/step - loss: 0.8042 - acc: 0.9557 - val_loss: 1.3144 - val_acc: 0.8360
Epoch 3/200
45000/45000 [==============================] - 28s 622us/step - loss: 0.7979 - acc: 0.9582 - val_loss: 1.2736 - val_acc: 0.8482
Epoch 4/200
45000/45000 [==============================] - 28s 620us/step - loss: 0.8083 - acc: 0.9570 - val_loss: 1.2634 - val_acc: 0.8592
Epoch 5/200
45000/45000 [==============================] - 28s 622us/step - loss: 0.8026 - acc: 0.9606 - val_loss: 1.3057 - val_acc: 0.8494
Epoch 6/200
45000/45000 [==============================] - 28s 623us/step - loss: 0.8116 - acc: 0.9593 - val_loss: 1.2727 - val_acc: 0.8568
Epoch 7/200
45000/45000 [==============================] - 28s 624us/step - loss: 0.8113 - acc: 0.9599 - val_lo

Epoch 59/200
45000/45000 [==============================] - 28s 621us/step - loss: 0.8049 - acc: 0.9629 - val_loss: 1.2964 - val_acc: 0.8508
Epoch 60/200
45000/45000 [==============================] - 28s 624us/step - loss: 0.8112 - acc: 0.9603 - val_loss: 1.2786 - val_acc: 0.8560
Epoch 61/200
45000/45000 [==============================] - 28s 626us/step - loss: 0.8046 - acc: 0.9628 - val_loss: 1.2977 - val_acc: 0.8550
Epoch 62/200
45000/45000 [==============================] - 28s 622us/step - loss: 0.8029 - acc: 0.9635 - val_loss: 1.3694 - val_acc: 0.8406
Epoch 63/200
45000/45000 [==============================] - 28s 625us/step - loss: 0.8010 - acc: 0.9632 - val_loss: 1.3098 - val_acc: 0.8512
Epoch 64/200
45000/45000 [==============================] - 28s 618us/step - loss: 0.8049 - acc: 0.9619 - val_loss: 1.2683 - val_acc: 0.8500
Epoch 65/200
45000/45000 [==============================] - 28s 623us/step - loss: 0.8072 - acc: 0.9624 - val_loss: 1.3419 - val_acc: 0.8380
Epoch 66/200


Epoch 117/200
45000/45000 [==============================] - 28s 623us/step - loss: 0.8061 - acc: 0.9625 - val_loss: 1.3264 - val_acc: 0.8368
Epoch 118/200
45000/45000 [==============================] - 28s 623us/step - loss: 0.7996 - acc: 0.9640 - val_loss: 1.4551 - val_acc: 0.8186
Epoch 119/200
45000/45000 [==============================] - 28s 624us/step - loss: 0.7961 - acc: 0.9650 - val_loss: 1.3355 - val_acc: 0.8374
Epoch 120/200
45000/45000 [==============================] - 28s 623us/step - loss: 0.7983 - acc: 0.9635 - val_loss: 1.2568 - val_acc: 0.8578
Epoch 121/200
45000/45000 [==============================] - 28s 622us/step - loss: 0.7924 - acc: 0.9651 - val_loss: 1.2913 - val_acc: 0.8498
Epoch 122/200
45000/45000 [==============================] - 28s 622us/step - loss: 0.7909 - acc: 0.9651 - val_loss: 1.3189 - val_acc: 0.8476
Epoch 123/200
45000/45000 [==============================] - 28s 625us/step - loss: 0.7931 - acc: 0.9640 - val_loss: 1.3030 - val_acc: 0.8482
Epoch 

Epoch 175/200
45000/45000 [==============================] - 28s 626us/step - loss: 0.7792 - acc: 0.9664 - val_loss: 1.2925 - val_acc: 0.8526
Epoch 176/200
45000/45000 [==============================] - 28s 625us/step - loss: 0.7872 - acc: 0.9640 - val_loss: 1.2890 - val_acc: 0.8462
Epoch 177/200
45000/45000 [==============================] - 28s 627us/step - loss: 0.7799 - acc: 0.9655 - val_loss: 1.2577 - val_acc: 0.8612
Epoch 178/200
45000/45000 [==============================] - 28s 621us/step - loss: 0.7894 - acc: 0.9626 - val_loss: 1.2736 - val_acc: 0.8576
Epoch 179/200
45000/45000 [==============================] - 28s 627us/step - loss: 0.7897 - acc: 0.9635 - val_loss: 1.2744 - val_acc: 0.8498
Epoch 180/200
45000/45000 [==============================] - 28s 625us/step - loss: 0.7736 - acc: 0.9682 - val_loss: 1.2821 - val_acc: 0.8538
Epoch 181/200
45000/45000 [==============================] - 28s 623us/step - loss: 0.7865 - acc: 0.9639 - val_loss: 1.3652 - val_acc: 0.8358
Epoch 

In [37]:
import functools
import keras
top1_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=1) #top-1精度

top1_acc.__name__ = 'top1_acc'

model_5.compile(loss='categorical_crossentropy',
             optimizer='adam',metrics=['accuracy','top_k_categorical_accuracy',top1_acc])

model_5.evaluate(x_test,y_test,verbose=0)


[1.3154940271377564, 0.8453, 0.9849, 0.8453]

In [38]:
score=model_5.evaluate(x_test,y_test,verbose=0)
print("测试损失为：{:.4f}".format(score[0]))
print("测试准确率为：{:.4f}".format(score[1]))

测试损失为：1.3155
测试准确率为：0.8453


In [39]:
tensor =[]
for i in range(0,256):
   
    u=model_5.get_layer('conv2d_6').get_weights()[0][:,:,:,i].squeeze()
    v=np.mean(u, axis=1)
    #print(v)
    vT=v.T
    D=np.cov(vT)
    try:                 
        invD=np.linalg.inv(D)
        #t=np.mean(v,axis=1)
        a=[]

        for j in range(0,2): #表示filter的大小
            for k in range(j+1,3):
               #if (j!=k) and (j<k):
                tp=v[j]-v[k]
                d=np.sqrt(abs(np.dot(np.dot(tp,invD),tp.T)))
                a.append(d)
            
        print(i,a)
        tensor.append(a)
    except:
        print("不可逆")
print(tensor)

0 [21.918681471526895, 11.391499678069886, 15.741208180902342]
1 [17.77950681914247, 6.837675420822669, 2.893745758514921]
2 [8.585788492952487, 20.290318864034322, 16.954327504628008]
3 [15.86003327906173, 20.45135134823226, 29.503923025976214]
4 [9.395418235479466, 12.356042772020855, 7.412684225073552]
5 [3.701248916991699, 11.461284234354332, 16.019895373180162]
6 [20.030045590665434, 16.10979845173103, 15.556390972732455]
7 [53.0005119227032, 24.616132349798434, 24.182863446575304]
8 [17.835181602471035, 8.703537580206378, 24.980996261884705]
9 [57.71618392188734, 45.46557721082984, 27.334348655854583]
10 [20.198488687406282, 3.416848030549089, 21.74163105972589]
11 [41.96038435033551, 28.429773819580866, 28.910772054438315]
12 [16.432390876708318, 21.16388983671911, 27.276076517792433]
13 [15.164852131601542, 20.244853978854035, 10.494594931198058]
14 [15.292354363856015, 12.400299990009279, 22.7148237653072]
15 [31.02882295857581, 5.067225758340191, 19.324429182576697]
16 [21.94

135 [31.986225799547814, 41.03506705316639, 23.573606428460312]
136 [23.53799200045174, 22.913539918233337, 45.040615402408044]
137 [11.591683627750832, 23.177560675916766, 26.586173963097693]
138 [11.412286869661969, 9.178470274392753, 19.207379309848406]
139 [64.01904013650939, 34.23404012679062, 35.05436839363773]
140 [33.71219767207566, 23.83556424939274, 22.538425398206993]
141 [12.432439186596408, 68.35197499009364, 28.02493801093073]
142 [16.24426862439895, 16.920586985197794, 11.501450322789204]
143 [19.71366200042912, 16.302192617746798, 1.9681499717293551]
144 [11.501190642212096, 36.38146726395625, 38.26725748738037]
145 [35.2969427357193, 21.532401974538068, 26.854488476427296]
146 [19.739142028653216, 22.04982063627061, 48.90060462421846]
147 [15.3641772752112, 11.049173636628586, 15.421564966735847]
148 [22.187173061917143, 7.482849706309154, 7.8891747832170065]
149 [16.412159522099248, 6.516182562301466, 37.394886658710085]
150 [44.18242943145032, 25.875523815289355, 29.

In [40]:
entt=[]
for i in range (0,256):
    
    data=tensor[i]
    data0=np.array(data)
    print('四舍五入，精确到个位\n',np.round(data0,decimals=-2))
    data1=np.round(data0,decimals=-1)

    data1_value_list=set([data1[i] for i in range (data1.shape[0])])
    print(data1_value_list)
    ent=0.0
    for data1_value in data1_value_list:
        p=float(data1[data1==data1_value].shape[0])/data1.shape[0]
        print(p)
        logp=np.log2(p)
        ent-=p*logp
        print(ent)   
    entt.append(ent)
print(entt)

e=np.mean(entt)
print(e)
bb=[k for k in range(len(entt)) if entt[k]<np.mean(entt)]
print(bb)
c=len(bb)
print(c)

四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{20.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{20.0}
1.0
0.0
四舍五入，精确到个位
 [100.   0.   0.]
{50.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [100.   0.   0.]
{50.0, 60.0, 30.0}
0

1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [200. 100.   0.]
{120.0, 150.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [100.   0. 100.]
{50.0, 60.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [100. 100. 100.]
{80.0, 70.0, 110.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{20.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个

{10.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
[0.9182958340544896, 1.584962500721156, 0.9182958340544896, 0.9182958340544896, 0.0, 1.584962500721156, 0.0, 0.9182958340544896, 0.9182958340544896, 1.584962500721156, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 1.584962500721156, 1.584962500721156, 1.584962500721156, 1.584962500721156, 0.9182958340544896, 1.584962500721156, 0.9182958340544896, 1.584962500721156, 0.9182958340544896, 1.584962500721156, 0.9182958340544896, 0.9182958340544896, 1.584962500721156, 0.9182958340544896, 0.9182958340544896, 0.9

In [41]:
from kerassurgeon.operations import delete_layer,insert_layer,delete_channels
model_6 = delete_channels(model_5, model_5.layers[21],bb)
model_6.summary()
model_6.save('ent_s_cifar10vgg16_6.h5')
model_6.save_weights('ent_s_cifar10vgg16_6_weights.h5',overwrite=True)

Deleting 161/256 channels from layer: conv2d_6
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 35)        980       
_________________________________________________________________
activation_1 (Activation)    multiple                  0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 35)        140       
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 18)        5688      
_________________________________________________________________
activation_2 (Activation)    

In [42]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model_6.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])
 
train_history=model_6.fit(x_train,y_train,epochs=200, batch_size=128,
             validation_split=0.1, verbose=1)

model_6.save('ent_s_cifar10vgg16_6.h5')
model_6.save_weights('ent_s_cifar10vgg16_6_weights.h5',overwrite=True)

Train on 45000 samples, validate on 5000 samples
Epoch 1/200
45000/45000 [==============================] - 38s 841us/step - loss: 0.9128 - acc: 0.9076 - val_loss: 1.2553 - val_acc: 0.8352
Epoch 2/200
45000/45000 [==============================] - 27s 607us/step - loss: 0.8679 - acc: 0.9304 - val_loss: 1.3402 - val_acc: 0.8222
Epoch 3/200
45000/45000 [==============================] - 26s 589us/step - loss: 0.8523 - acc: 0.9404 - val_loss: 1.3335 - val_acc: 0.8348
Epoch 4/200
45000/45000 [==============================] - 27s 608us/step - loss: 0.8564 - acc: 0.9424 - val_loss: 1.3252 - val_acc: 0.8338
Epoch 5/200
45000/45000 [==============================] - 27s 608us/step - loss: 0.8554 - acc: 0.9450 - val_loss: 1.2803 - val_acc: 0.8498
Epoch 6/200
45000/45000 [==============================] - 27s 608us/step - loss: 0.8505 - acc: 0.9483 - val_loss: 1.2329 - val_acc: 0.8556
Epoch 7/200
45000/45000 [==============================] - 27s 607us/step - loss: 0.8435 - acc: 0.9516 - val_lo

Epoch 59/200
45000/45000 [==============================] - 27s 610us/step - loss: 0.8252 - acc: 0.9570 - val_loss: 1.3359 - val_acc: 0.8434
Epoch 60/200
45000/45000 [==============================] - 27s 607us/step - loss: 0.8217 - acc: 0.9580 - val_loss: 1.3121 - val_acc: 0.8424
Epoch 61/200
45000/45000 [==============================] - 27s 604us/step - loss: 0.8188 - acc: 0.9592 - val_loss: 1.3049 - val_acc: 0.8486
Epoch 62/200
45000/45000 [==============================] - 27s 609us/step - loss: 0.8260 - acc: 0.9572 - val_loss: 1.2913 - val_acc: 0.8506
Epoch 63/200
45000/45000 [==============================] - 27s 609us/step - loss: 0.8254 - acc: 0.9571 - val_loss: 1.5545 - val_acc: 0.7962
Epoch 64/200
45000/45000 [==============================] - 27s 610us/step - loss: 0.8268 - acc: 0.9576 - val_loss: 1.3066 - val_acc: 0.8420
Epoch 65/200
45000/45000 [==============================] - 27s 611us/step - loss: 0.8172 - acc: 0.9586 - val_loss: 1.2816 - val_acc: 0.8506
Epoch 66/200


Epoch 117/200
45000/45000 [==============================] - 27s 609us/step - loss: 0.8155 - acc: 0.9586 - val_loss: 1.5191 - val_acc: 0.7924
Epoch 118/200
45000/45000 [==============================] - 27s 606us/step - loss: 0.8179 - acc: 0.9575 - val_loss: 1.3255 - val_acc: 0.8416
Epoch 119/200
45000/45000 [==============================] - 27s 610us/step - loss: 0.8140 - acc: 0.9582 - val_loss: 1.3164 - val_acc: 0.8450
Epoch 120/200
45000/45000 [==============================] - 27s 610us/step - loss: 0.8186 - acc: 0.9576 - val_loss: 1.2708 - val_acc: 0.8534
Epoch 121/200
45000/45000 [==============================] - 27s 611us/step - loss: 0.8115 - acc: 0.9592 - val_loss: 1.3134 - val_acc: 0.8476
Epoch 122/200
45000/45000 [==============================] - 27s 609us/step - loss: 0.7979 - acc: 0.9623 - val_loss: 1.3184 - val_acc: 0.8484
Epoch 123/200
45000/45000 [==============================] - 27s 609us/step - loss: 0.8060 - acc: 0.9600 - val_loss: 1.4040 - val_acc: 0.8380
Epoch 

Epoch 175/200
45000/45000 [==============================] - 27s 608us/step - loss: 0.7995 - acc: 0.9615 - val_loss: 1.2965 - val_acc: 0.8458
Epoch 176/200
45000/45000 [==============================] - 28s 613us/step - loss: 0.7979 - acc: 0.9608 - val_loss: 1.2738 - val_acc: 0.8570
Epoch 177/200
45000/45000 [==============================] - 27s 608us/step - loss: 0.8109 - acc: 0.9577 - val_loss: 1.3113 - val_acc: 0.8454
Epoch 178/200
45000/45000 [==============================] - 28s 612us/step - loss: 0.8106 - acc: 0.9583 - val_loss: 1.3427 - val_acc: 0.8384
Epoch 179/200
45000/45000 [==============================] - 27s 606us/step - loss: 0.8059 - acc: 0.9594 - val_loss: 1.3841 - val_acc: 0.8258
Epoch 180/200
45000/45000 [==============================] - 27s 609us/step - loss: 0.8008 - acc: 0.9610 - val_loss: 1.3451 - val_acc: 0.8386
Epoch 181/200
45000/45000 [==============================] - 27s 608us/step - loss: 0.8003 - acc: 0.9606 - val_loss: 1.2732 - val_acc: 0.8580
Epoch 

In [43]:
import functools
import keras
top1_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=1) #top-1精度

top1_acc.__name__ = 'top1_acc'

model_6.compile(loss='categorical_crossentropy',
             optimizer='adam',metrics=['accuracy','top_k_categorical_accuracy',top1_acc])

model_6.evaluate(x_test,y_test,verbose=0)


[1.516072407913208, 0.8029, 0.9809, 0.8029]

In [44]:
score=model_6.evaluate(x_test,y_test,verbose=0)
print("测试损失为：{:.4f}".format(score[0]))
print("测试准确率为：{:.4f}".format(score[1]))

测试损失为：1.5161
测试准确率为：0.8029


In [45]:
tensor =[]
for i in range(0,256):
   
    u=model_6.get_layer('conv2d_7').get_weights()[0][:,:,:,i].squeeze()
    v=np.mean(u, axis=1)
    #print(v)
    vT=v.T
    D=np.cov(vT)
    try:                 
        invD=np.linalg.inv(D)
        #t=np.mean(v,axis=1)
        a=[]

        for j in range(0,2): #表示filter的大小
            for k in range(j+1,3):
               #if (j!=k) and (j<k):
                tp=v[j]-v[k]
                d=np.sqrt(abs(np.dot(np.dot(tp,invD),tp.T)))
                a.append(d)
            
        print(i,a)
        tensor.append(a)
    except:
        print("不可逆")
print(tensor)

0 [68.11593506024019, 61.32247759626907, 99.12569755318383]
1 [27.907579311308385, 9.531733816405804, 21.340819932653694]
2 [5.130739022576148, 8.316888740558587, 8.72823520567606]
3 [30.95760251838833, 23.926521831650525, 16.211235701794173]
4 [11.993985715361742, 5.864554270191802, 8.130010057891218]
5 [40.26524876872851, 23.946112893497556, 27.041760027645413]
6 [8.927656074469311, 16.799934569867194, 9.117684711146921]
7 [63.685592607830436, 28.294425668774075, 177.96637073832105]
8 [11.543743973726807, 13.496299960690587, 6.553201572914961]
9 [8.969071850999574, 2.910972777647883, 11.358086220359104]
10 [12.100463186809304, 7.843313744560106, 27.21637898573605]
11 [9.163624101772248, 8.264338748416469, 4.298140960082398]
12 [33.10930944258936, 26.94910250073447, 14.015486538988783]
13 [20.49088922271787, 43.44498832429406, 8.90185071476256]
14 [17.65813056585752, 16.195025657251897, 10.393987860053942]
15 [8.292562107317071, 17.08749336516022, 9.243198265338858]
16 [13.22839223842

154 [16.408964415749296, 10.288374916222097, 9.840889481799868]
155 [13.332062938886123, 1.4516307983098684, 18.315235285183636]
156 [16.93077065629613, 20.969467560135, 14.536788631428486]
157 [11.862139232684724, 27.335324097647113, 18.65751014939213]
158 [17.995855900770263, 24.25975779229291, 7.285377609530761]
159 [8.457113287618117, 15.846949989654457, 1.5689060498319067]
160 [8.719971041478185, 15.302099019720538, 4.157646002118346]
161 [8.526611939787246, 5.903737093693092, 14.875837991655445]
162 [7.65376543486137, 10.695966852427414, 7.627264827752494]
163 [68.24118201612846, 26.108749236045085, 72.61117479898309]
164 [19.781003501525372, 36.4505673985739, 41.96578780433912]
165 [19.550935489682548, 12.752278741391459, 22.8862770508026]
166 [9.991263477350234, 9.815456734581025, 3.889900477099732]
167 [13.739671096155128, 6.949823242793956, 9.117406219761598]
168 [23.827288103259104, 15.20202278305332, 20.570858030695234]
169 [40.364819604325724, 8.940223118944175, 19.2357469

In [49]:
entt=[]
for i in range (0,256):
    
    data=tensor[i]
    data0=np.array(data)
    print('四舍五入，精确到个位\n',np.round(data0,decimals=-2))
    data1=np.round(data0,decimals=-1)

    data1_value_list=set([data1[i] for i in range (data1.shape[0])])
    print(data1_value_list)
    ent=0.0
    for data1_value in data1_value_list:
        p=float(data1[data1==data1_value].shape[0])/data1.shape[0]
        print(p)
        logp=np.log2(p)
        ent-=p*logp
        print(ent)   
    entt.append(ent)
print(entt)

e=np.mean(entt)
print(e)
bb=[k for k in range(len(entt)) if entt[k]<np.mean(entt)]
print(bb)
c=len(bb)
print(c)

四舍五入，精确到个位
 [100. 100. 100.]
{100.0, 60.0, 70.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{20.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [100.   0. 200.]
{180.0, 60.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{0.

{40.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{20.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 30.0}
0.3333333333333333
0.5283208335737187
0.6

In [50]:
from kerassurgeon.operations import delete_layer,insert_layer,delete_channels
model_7 = delete_channels(model_6, model_6.layers[25],bb)
model_7.summary()
model_7.save('ent_s_cifar10vgg16_7.h5')
model_7.save_weights('ent_s_cifar10vgg16_7_weights.h5',overwrite=True)

Deleting 189/256 channels from layer: conv2d_7
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 35)        980       
_________________________________________________________________
activation_1 (Activation)    multiple                  0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 35)        140       
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 18)        5688      
_________________________________________________________________
activation_2 (Activation)    

In [51]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model_7.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])
 
train_history=model_7.fit(x_train,y_train,epochs=200, batch_size=128,
             validation_split=0.1, verbose=1)

model_7.save('ent_b_cifar10vgg16_7.h5')
model_7.save_weights('ent_b_cifar10vgg16_7_weights.h5',overwrite=True)

Train on 45000 samples, validate on 5000 samples
Epoch 1/200
45000/45000 [==============================] - 38s 845us/step - loss: 1.1069 - acc: 0.8403 - val_loss: 1.1610 - val_acc: 0.8342
Epoch 2/200
45000/45000 [==============================] - 27s 605us/step - loss: 0.9423 - acc: 0.8990 - val_loss: 1.2596 - val_acc: 0.8162
Epoch 3/200
45000/45000 [==============================] - 27s 603us/step - loss: 0.9082 - acc: 0.9139 - val_loss: 1.2767 - val_acc: 0.8270
Epoch 4/200
45000/45000 [==============================] - 27s 607us/step - loss: 0.8901 - acc: 0.9252 - val_loss: 1.2259 - val_acc: 0.8376
Epoch 5/200
45000/45000 [==============================] - 27s 605us/step - loss: 0.8810 - acc: 0.9287 - val_loss: 1.2302 - val_acc: 0.8496
Epoch 6/200
45000/45000 [==============================] - 27s 605us/step - loss: 0.8696 - acc: 0.9349 - val_loss: 1.2894 - val_acc: 0.8364
Epoch 7/200
45000/45000 [==============================] - 27s 604us/step - loss: 0.8660 - acc: 0.9371 - val_lo

Epoch 59/200
45000/45000 [==============================] - 27s 601us/step - loss: 0.8183 - acc: 0.9553 - val_loss: 1.4216 - val_acc: 0.8160
Epoch 60/200
45000/45000 [==============================] - 27s 603us/step - loss: 0.8336 - acc: 0.9505 - val_loss: 1.4665 - val_acc: 0.8132
Epoch 61/200
45000/45000 [==============================] - 27s 606us/step - loss: 0.8256 - acc: 0.9540 - val_loss: 1.3874 - val_acc: 0.8268
Epoch 62/200
45000/45000 [==============================] - 27s 603us/step - loss: 0.8150 - acc: 0.9565 - val_loss: 1.4464 - val_acc: 0.8078
Epoch 63/200
45000/45000 [==============================] - 27s 604us/step - loss: 0.8221 - acc: 0.9542 - val_loss: 1.3201 - val_acc: 0.8386
Epoch 64/200
45000/45000 [==============================] - 27s 605us/step - loss: 0.8272 - acc: 0.9520 - val_loss: 1.3364 - val_acc: 0.8376
Epoch 65/200
45000/45000 [==============================] - 27s 602us/step - loss: 0.8319 - acc: 0.9520 - val_loss: 1.4195 - val_acc: 0.8224
Epoch 66/200


Epoch 117/200
45000/45000 [==============================] - 27s 606us/step - loss: 0.8091 - acc: 0.9553 - val_loss: 1.3614 - val_acc: 0.8316
Epoch 118/200
45000/45000 [==============================] - 27s 604us/step - loss: 0.8074 - acc: 0.9566 - val_loss: 1.3077 - val_acc: 0.8448
Epoch 119/200
45000/45000 [==============================] - 27s 606us/step - loss: 0.8088 - acc: 0.9558 - val_loss: 1.3694 - val_acc: 0.8346
Epoch 120/200
45000/45000 [==============================] - 27s 605us/step - loss: 0.8053 - acc: 0.9577 - val_loss: 1.3698 - val_acc: 0.8250
Epoch 121/200
45000/45000 [==============================] - 27s 601us/step - loss: 0.7994 - acc: 0.9572 - val_loss: 1.4995 - val_acc: 0.8004
Epoch 122/200
45000/45000 [==============================] - 27s 605us/step - loss: 0.8024 - acc: 0.9559 - val_loss: 1.4156 - val_acc: 0.8278
Epoch 123/200
45000/45000 [==============================] - 27s 606us/step - loss: 0.8050 - acc: 0.9564 - val_loss: 1.3237 - val_acc: 0.8364
Epoch 

Epoch 175/200
45000/45000 [==============================] - 27s 607us/step - loss: 0.7936 - acc: 0.9572 - val_loss: 1.2645 - val_acc: 0.8484
Epoch 176/200
45000/45000 [==============================] - 27s 607us/step - loss: 0.7929 - acc: 0.9581 - val_loss: 1.3013 - val_acc: 0.8338
Epoch 177/200
45000/45000 [==============================] - 27s 603us/step - loss: 0.7831 - acc: 0.9602 - val_loss: 1.2923 - val_acc: 0.8472
Epoch 178/200
45000/45000 [==============================] - 27s 604us/step - loss: 0.7944 - acc: 0.9568 - val_loss: 1.2991 - val_acc: 0.8366
Epoch 179/200
45000/45000 [==============================] - 27s 602us/step - loss: 0.7913 - acc: 0.9558 - val_loss: 1.3224 - val_acc: 0.8430
Epoch 180/200
45000/45000 [==============================] - 27s 607us/step - loss: 0.8034 - acc: 0.9548 - val_loss: 1.3177 - val_acc: 0.8364
Epoch 181/200
45000/45000 [==============================] - 27s 604us/step - loss: 0.7956 - acc: 0.9569 - val_loss: 1.3247 - val_acc: 0.8396
Epoch 

In [52]:
import functools
import keras
top1_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=1) #top-1精度

top1_acc.__name__ = 'top1_acc'

model_7.compile(loss='categorical_crossentropy',
             optimizer='adam',metrics=['accuracy','top_k_categorical_accuracy',top1_acc])

model_7.evaluate(x_test,y_test,verbose=0)


[1.354774484062195, 0.8301, 0.9862, 0.8301]

In [53]:
score=model_7.evaluate(x_test,y_test,verbose=0)
print("测试损失为：{:.4f}".format(score[0]))
print("测试准确率为：{:.4f}".format(score[1]))

测试损失为：1.3548
测试准确率为：0.8301


In [54]:
tensor =[]
for i in range(0,512):
   
    u=model_7.get_layer('conv2d_8').get_weights()[0][:,:,:,i].squeeze()
    v=np.mean(u, axis=1)
    #print(v)
    vT=v.T
    D=np.cov(vT)
    try:                 
        invD=np.linalg.inv(D)
        #t=np.mean(v,axis=1)
        a=[]

        for j in range(0,2): #表示filter的大小
            for k in range(j+1,3):
               #if (j!=k) and (j<k):
                tp=v[j]-v[k]
                d=np.sqrt(abs(np.dot(np.dot(tp,invD),tp.T)))
                a.append(d)
            
        print(i,a)
        tensor.append(a)
    except:
        print("不可逆")
print(tensor)

0 [12.831050822574385, 3.8694715440203873, 4.073792827576552]
1 [19.26948076879917, 22.140490954847404, 18.084647357973445]
2 [11.298205623112196, 49.951199749503466, 30.676178624921263]
3 [6.822059608579291, 9.347817323526781, 26.000232328697376]
4 [11.12075604702185, 6.0005191439309, 14.14903616099435]
5 [24.956084905317624, 15.308875343904154, 4.4300914089123795]
6 [7.73708071150386, 4.9726581918093835, 3.7435784990006464]
7 [6.306689968961622, 7.914064036422836, 26.783317432703484]
8 [30.605567153938008, 26.069795483934524, 16.121186034537168]
9 [8.060094001236065, 3.313867538595122, 12.601200502970277]
10 [27.028565131238263, 15.142706843055047, 36.14056430783517]
11 [9.857414812478895, 1.4605991563318441, 3.6960048228620463]
12 [18.082220933981933, 31.296511102809077, 6.939328828960563]
13 [24.85880653407528, 8.969645279384702, 14.337770420740302]
14 [15.28000902625012, 8.109293266575405, 14.647561171703929]
15 [15.224369907025158, 18.76649700492384, 21.648146430171618]
16 [9.448

156 [33.77648248475511, 12.442072851986264, 9.507210805719117]
157 [7.163403887410878, 5.399516548485331, 14.314056178619252]
158 [11.814939317680597, 9.090041017051623, 14.167255645551036]
159 [15.297294704458029, 20.422047831030927, 10.737747129143237]
160 [10.043382170991338, 15.26318945924711, 10.999469657597729]
161 [27.217525345135364, 31.052881912427043, 23.0327783126686]
162 [10.928415743914782, 1.9839856223808283, 13.770236495334272]
163 [15.018583548104056, 16.733859028417225, 17.71615506232266]
164 [13.864372685544922, 50.13498970619797, 6.257043150662924]
165 [13.504032283390131, 22.773582162283084, 7.9734006736961955]
166 [6.466961147995048, 23.34656159450097, 11.645371865812564]
167 [9.760561968774597, 2.3315664368092475, 20.475935674953455]
168 [12.362503408155753, 14.39980683462072, 22.24397448056408]
169 [9.701941855716992, 10.394471659699185, 13.922303849282299]
170 [22.649262185953305, 16.457268565377582, 8.398461615949206]
171 [23.160104407633515, 41.35954636847053,

326 [7.94548739650582, 12.59061137198901, 12.429250487695958]
327 [12.835493956580148, 9.386627514316805, 37.997708753475074]
328 [12.438647116664075, 18.857377396091056, 10.008039485313775]
329 [8.031573641597767, 18.829767837713895, 12.67749491482924]
330 [19.417658893735158, 10.892539328153266, 13.088916077465955]
331 [8.843853953259407, 6.664405304118001, 1.1790919537458406]
332 [24.729455223638855, 20.521742757664796, 43.40945692965686]
333 [38.028614012393525, 16.28710108961633, 72.46250466844702]
334 [14.987584061820366, 15.782291793428788, 13.390278067261987]
335 [88.45073833254455, 53.188286637653775, 76.97217463294518]
336 [11.912549732660832, 13.845686912661659, 11.345894350767347]
337 [70.49975937606114, 68.08959895064744, 80.99375787866381]
338 [73.63328487497684, 33.048522475279775, 26.428185717127402]
339 [4.443730989783035, 10.346993251943964, 20.840434003656316]
340 [12.829874548978855, 10.58981709352049, 12.63257832212796]
341 [8.526717802132504, 7.493949929466377, 9.

495 [13.402819410181474, 15.096764226028137, 9.160259059174525]
496 [9.871019913367435, 11.08460957246959, 11.558846963711094]
497 [20.53098690613395, 12.547614335905193, 7.944742382979207]
498 [24.28681053404178, 19.004590154497468, 22.37219281941303]
499 [23.447035454474698, 11.719459247743016, 10.192072181686722]
500 [3.0932220625047773, 15.09983175575556, 9.406445447342312]
501 [24.18491062823666, 6.583349694662245, 11.656979862711152]
502 [5.824384250758373, 35.4179533212354, 13.445966173825378]
503 [17.0690459665391, 11.839049843475154, 14.556526809097974]
504 [12.581947162942905, 7.187924015972436, 6.494724076527864]
505 [10.27448668253477, 6.857307417915847, 11.78268937847226]
506 [9.284321697317067, 12.641001465876931, 17.198738945708545]
507 [21.754554173086262, 50.54181692295534, 12.479931476233723]
508 [2.878152528645661, 12.748966997972065, 39.60883747766901]
509 [14.373884597295422, 11.454522095801087, 12.71723230665492]
510 [9.50429274795792, 3.147807396625467, 17.158293

In [55]:
entt=[]
for i in range (0,512):
    
    data=tensor[i]
    data0=np.array(data)
    print('四舍五入，精确到个位\n',np.round(data0,decimals=-2))
    data1=np.round(data0,decimals=-1)

    data1_value_list=set([data1[i] for i in range (data1.shape[0])])
    print(data1_value_list)
    ent=0.0
    for data1_value in data1_value_list:
        p=float(data1[data1==data1_value].shape[0])/data1.shape[0]
        print(p)
        logp=np.log2(p)
        ent-=p*logp
        print(ent)   
    entt.append(ent)
print(entt)

e=np.mean(entt)
print(e)
bb=[k for k in range(len(entt)) if entt[k]<np.mean(entt)]
print(bb)
c=len(bb)
print(c)

四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{20.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 50.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.91

{0.0, 40.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 30.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [100.   0.   0.]
{80.0, 30.0}


0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [100.   0. 100.]
{50.0, 10.0, 60.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 10.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 50.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500

 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 30.0}
0.6666666666666666
0.389975

 [0. 0. 0.]
{40.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [100.   0. 100.]
{40.0, 60.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]


In [56]:
from kerassurgeon.operations import delete_layer,insert_layer,delete_channels
model_8 = delete_channels(model_7, model_7.layers[29],bb)
model_8.summary()
model_8.save('ent_s_cifar10vgg16_8.h5')
model_8.save_weights('ent_s_cifar10vgg16_8_weights.h5',overwrite=True)

Deleting 114/512 channels from layer: conv2d_8
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 35)        980       
_________________________________________________________________
activation_1 (Activation)    multiple                  0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 35)        140       
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 18)        5688      
_________________________________________________________________
activation_2 (Activation)    

In [57]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model_8.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])
 
train_history=model_8.fit(x_train,y_train,epochs=200, batch_size=128,
             validation_split=0.1, verbose=1)

model_8.save('ent_s_cifar10vgg16_8.h5')
model_8.save_weights('ent_s_cifar10vgg16_8_weights.h5',overwrite=True)

Train on 45000 samples, validate on 5000 samples
Epoch 1/200
45000/45000 [==============================] - 38s 842us/step - loss: 0.7854 - acc: 0.9448 - val_loss: 1.2905 - val_acc: 0.8304
Epoch 2/200
45000/45000 [==============================] - 26s 578us/step - loss: 0.8096 - acc: 0.9428 - val_loss: 1.2833 - val_acc: 0.8382
Epoch 3/200
45000/45000 [==============================] - 26s 583us/step - loss: 0.8131 - acc: 0.9453 - val_loss: 1.3198 - val_acc: 0.8298
Epoch 4/200
45000/45000 [==============================] - 26s 583us/step - loss: 0.8186 - acc: 0.9458 - val_loss: 1.3078 - val_acc: 0.8332
Epoch 5/200
45000/45000 [==============================] - 26s 585us/step - loss: 0.8227 - acc: 0.9480 - val_loss: 1.3053 - val_acc: 0.8416
Epoch 6/200
45000/45000 [==============================] - 26s 583us/step - loss: 0.8239 - acc: 0.9487 - val_loss: 1.3140 - val_acc: 0.8306
Epoch 7/200
45000/45000 [==============================] - 26s 585us/step - loss: 0.8201 - acc: 0.9490 - val_lo

Epoch 59/200
45000/45000 [==============================] - 26s 587us/step - loss: 0.8115 - acc: 0.9535 - val_loss: 1.3179 - val_acc: 0.8396
Epoch 60/200
45000/45000 [==============================] - 27s 589us/step - loss: 0.8165 - acc: 0.9514 - val_loss: 1.3467 - val_acc: 0.8318
Epoch 61/200
45000/45000 [==============================] - 26s 583us/step - loss: 0.8115 - acc: 0.9556 - val_loss: 1.3254 - val_acc: 0.8404
Epoch 62/200
45000/45000 [==============================] - 26s 584us/step - loss: 0.8113 - acc: 0.9548 - val_loss: 1.3810 - val_acc: 0.8224
Epoch 63/200
45000/45000 [==============================] - 26s 584us/step - loss: 0.8195 - acc: 0.9512 - val_loss: 1.4291 - val_acc: 0.8204
Epoch 64/200
45000/45000 [==============================] - 26s 584us/step - loss: 0.8123 - acc: 0.9538 - val_loss: 1.3402 - val_acc: 0.8358
Epoch 65/200
45000/45000 [==============================] - 26s 584us/step - loss: 0.8137 - acc: 0.9534 - val_loss: 1.3855 - val_acc: 0.8286
Epoch 66/200


Epoch 117/200
45000/45000 [==============================] - 26s 585us/step - loss: 0.7887 - acc: 0.9572 - val_loss: 1.3395 - val_acc: 0.8378
Epoch 118/200
45000/45000 [==============================] - 27s 589us/step - loss: 0.7941 - acc: 0.9558 - val_loss: 1.3363 - val_acc: 0.8348
Epoch 119/200
45000/45000 [==============================] - 26s 587us/step - loss: 0.7974 - acc: 0.9549 - val_loss: 1.3751 - val_acc: 0.8286
Epoch 120/200
45000/45000 [==============================] - 26s 581us/step - loss: 0.7881 - acc: 0.9573 - val_loss: 1.3708 - val_acc: 0.8298
Epoch 121/200
45000/45000 [==============================] - 26s 586us/step - loss: 0.7823 - acc: 0.9583 - val_loss: 1.3497 - val_acc: 0.8344
Epoch 122/200
45000/45000 [==============================] - 26s 588us/step - loss: 0.7872 - acc: 0.9558 - val_loss: 1.3131 - val_acc: 0.8346
Epoch 123/200
45000/45000 [==============================] - 26s 586us/step - loss: 0.7925 - acc: 0.9555 - val_loss: 1.3064 - val_acc: 0.8414
Epoch 

Epoch 175/200
45000/45000 [==============================] - 26s 581us/step - loss: 0.7744 - acc: 0.9587 - val_loss: 1.3295 - val_acc: 0.8394
Epoch 176/200
45000/45000 [==============================] - 26s 586us/step - loss: 0.7873 - acc: 0.9544 - val_loss: 1.3423 - val_acc: 0.8342
Epoch 177/200
45000/45000 [==============================] - 26s 589us/step - loss: 0.7970 - acc: 0.9533 - val_loss: 1.3109 - val_acc: 0.8298
Epoch 178/200
45000/45000 [==============================] - 26s 587us/step - loss: 0.7884 - acc: 0.9556 - val_loss: 1.3164 - val_acc: 0.8414
Epoch 179/200
45000/45000 [==============================] - 26s 584us/step - loss: 0.7786 - acc: 0.9596 - val_loss: 1.2954 - val_acc: 0.8368
Epoch 180/200
45000/45000 [==============================] - 26s 581us/step - loss: 0.7816 - acc: 0.9572 - val_loss: 1.2813 - val_acc: 0.8428
Epoch 181/200
45000/45000 [==============================] - 26s 587us/step - loss: 0.7755 - acc: 0.9593 - val_loss: 1.4371 - val_acc: 0.8148
Epoch 

In [58]:
import functools
import keras
top1_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=1) #top-1精度

top1_acc.__name__ = 'top1_acc'

model_8.compile(loss='categorical_crossentropy',
             optimizer='adam',metrics=['accuracy','top_k_categorical_accuracy',top1_acc])

model_8.evaluate(x_test,y_test,verbose=0)


[1.320778325843811, 0.8307, 0.9889, 0.8307]

In [59]:
score=model_8.evaluate(x_test,y_test,verbose=0)
print("测试损失为：{:.4f}".format(score[0]))
print("测试准确率为：{:.4f}".format(score[1]))

测试损失为：1.3208
测试准确率为：0.8307


In [60]:
tensor =[]
for i in range(0,512):
   
    u=model_8.get_layer('conv2d_9').get_weights()[0][:,:,:,i].squeeze()
    v=np.mean(u, axis=1)
    #print(v)
    vT=v.T
    D=np.cov(vT)
    try:                 
        invD=np.linalg.inv(D)
        #t=np.mean(v,axis=1)
        a=[]

        for j in range(0,2): #表示filter的大小
            for k in range(j+1,3):
               #if (j!=k) and (j<k):
                tp=v[j]-v[k]
                d=np.sqrt(abs(np.dot(np.dot(tp,invD),tp.T)))
                a.append(d)
            
        print(i,a)
        tensor.append(a)
    except:
        print("不可逆")
print(tensor)

0 [12.333804989293288, 86.63823164032549, 97.6738274024738]
1 [8.274944856119959, 23.44807658288541, 12.931437430998528]
2 [10.091335137673303, 7.629620262285646, 93.94860985312641]
3 [16.764546500977307, 25.163037965863307, 15.072284575387773]
4 [60.6877038797083, 125.69367317052063, 83.52838397006076]
5 [41.300522099443796, 14.209582263650553, 35.11874434400989]
6 [12.126992445378354, 38.58687112022055, 20.099148465618356]
7 [8.010436373431334, 10.928478029530515, 13.2184726695014]
8 [26.96240414322538, 23.514704871948613, 61.42127498623794]
9 [17.80706685509982, 20.12700427482432, 10.265993190067514]
10 [21.31726934815293, 31.174995774771276, 10.38918607686775]
11 [53.73389161056944, 48.341857754761975, 91.81160569972681]
12 [12.753039427855523, 76.3034369783249, 79.98158586292496]
13 [19.168376724693346, 11.420151524381966, 37.806254124842205]
14 [13.289401786191267, 15.577763341839784, 17.97322419564045]
15 [16.047524948849517, 37.97787973661695, 21.406702167719548]
16 [22.4216558

137 [24.225191187620837, 11.795758690974614, 19.95948354423684]
138 [12.445820014409254, 11.617545537299101, 40.79339262155745]
139 [34.41464979036517, 4.499292847913616, 17.005487790423253]
140 [27.875238755405356, 20.35358253162714, 10.868677954410305]
141 [14.60501898628837, 22.49093801480153, 20.377484205256465]
142 [32.704300598215035, 38.57422520272889, 28.555509751114514]
143 [15.381028861913405, 15.72388568502118, 16.042111782817845]
144 [11.359352038332728, 28.24175273724187, 13.732435174077528]
145 [37.03441225867119, 44.018352322486166, 32.30437185739841]
146 [38.28276737864864, 25.13516697691741, 45.82736664283415]
147 [48.35010570131916, 12.383621064270828, 22.510696496478996]
148 [17.386623501436404, 7.014878808930308, 12.994144973513336]
149 [2.8177275501284593, 32.803936405763544, 40.44882007146371]
150 [23.48146358877174, 21.414031859928826, 18.150660443966068]
151 [27.731924751469066, 33.88259622347824, 20.21316059568839]
152 [25.277394117394476, 60.056844359684426, 3

271 [12.636121930941204, 16.99849009820862, 21.95623440154206]
272 [18.776158942404383, 28.928707834541143, 10.633705185773008]
273 [18.92951821044088, 3.115222435811761, 6.835176244582863]
274 [9.135027372938298, 17.25261657330529, 26.603685329616575]
275 [57.7958570454126, 18.89846000654277, 48.28195413119236]
276 [21.571843563732777, 7.627945440508315, 25.089418144245045]
277 [16.65186634314487, 10.965964439097881, 13.378714927920342]
278 [17.59504888696321, 24.962739949226428, 28.742960092830927]
279 [11.639823527555068, 15.625921786774736, 14.272877148182044]
280 [35.00171850617652, 31.083547265865143, 17.133869602974]
281 [7.954259630369047, 8.228071952790808, 28.902247812705056]
282 [17.145039437672224, 24.012610554676442, 8.519565139696107]
283 [19.759028363026637, 11.451856805759562, 11.392790286375782]
284 [10.536067234435944, 19.61557548338028, 20.164804651140372]
285 [27.78353069199472, 12.991567279190008, 14.442187089268986]
286 [89.54496002412786, 49.96512348692312, 105.6

402 [27.850436185985746, 8.748769619131185, 24.72291809512347]
403 [22.083392552230414, 20.251676195730685, 20.179158001848695]
404 [6.23401606394118, 15.750999540068257, 8.511321045826516]
405 [33.812016651132765, 24.356888101513594, 7.490744060895817]
406 [13.87839239765229, 10.579956437077103, 23.78885046953693]
407 [12.24381263331331, 4.42141564884758, 20.403270907062186]
408 [5.136569942741203, 16.769373493354458, 12.057659067223476]
409 [15.504995248714483, 24.08433204172871, 29.42438923794187]
410 [22.274866856392233, 20.10106783160751, 4.745263096838292]
411 [20.127585108800602, 19.05318706215162, 10.524890281123481]
412 [43.8222437320972, 51.48863783743925, 47.4064185605356]
413 [24.76370141293638, 15.609667831591487, 27.777645121787465]
414 [6.7558871839645205, 9.219181580866932, 1.8926985537987178]
415 [26.589203747393604, 44.00107087652811, 32.6929680825761]
416 [12.51936577793216, 12.44526332311499, 5.333098118284118]
417 [24.174449859955327, 6.4452432524689165, 27.9091483

In [61]:
entt=[]
for i in range (0,512):
    
    data=tensor[i]
    data0=np.array(data)
    print('四舍五入，精确到个位\n',np.round(data0,decimals=-2))
    data1=np.round(data0,decimals=-1)

    data1_value_list=set([data1[i] for i in range (data1.shape[0])])
    print(data1_value_list)
    ent=0.0
    for data1_value in data1_value_list:
        p=float(data1[data1==data1_value].shape[0])/data1.shape[0]
        print(p)
        logp=np.log2(p)
        ent-=p*logp
        print(ent)   
    entt.append(ent)
print(entt)

e=np.mean(entt)
print(e)
bb=[k for k in range(len(entt)) if entt[k]<np.mean(entt)]
print(bb)
c=len(bb)
print(c)

四舍五入，精确到个位
 [  0. 100. 100.]
{10.0, 100.0, 90.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [  0.   0. 100.]
{10.0, 90.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{20.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [100. 100. 100.]
{80.0, 130.0, 60.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 10.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [  0.   0. 10

{70.0, 100.0, 150.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{20.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{20.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{20.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{50.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566

0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{20.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 50.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [  0.   0. 100.]
{0.0, 60.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [100.   0.   0.]
{40.0, 60.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0, 30.0

0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [100.   0.   0.]
{40.0, 50.0, 60.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [100.   0.   0.]
{50.0, 60.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0, 30.0}
0.3333333333333333
0.52

四舍五入，精确到个位
 [100.   0.   0.]
{40.0, 50.0, 140.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [  0.   0. 100.]
{40.0, 50.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [  0. 100.   0.]
{40.0, 60.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333


0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 30.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{20.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [100.   0. 100.]
{90.0, 60.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [100.   0.   0.]
{10.0, 20.0, 70.0}
0.3333333333333333
0.5283208335737187
0.

In [62]:
from kerassurgeon.operations import delete_layer,insert_layer,delete_channels
model_9 = delete_channels(model_8, model_8.layers[33],bb)
model_9.summary()
model_9.save('ent_s_cifar10vgg16_9.h5')
model_9.save_weights('ent_s_cifar10vgg16_9_weights.h5',overwrite=True)

Deleting 317/512 channels from layer: conv2d_9
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 35)        980       
_________________________________________________________________
activation_1 (Activation)    multiple                  0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 35)        140       
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 18)        5688      
_________________________________________________________________
activation_2 (Activation)    

In [63]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model_9.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])
 
train_history=model_9.fit(x_train,y_train,epochs=200, batch_size=128,
             validation_split=0.1, verbose=1)

model_9.save('ent_s_cifar10vgg16_9.h5')
model_9.save_weights('ent_s_cifar10vgg16_9_weights.h5',overwrite=True)

Train on 45000 samples, validate on 5000 samples
Epoch 1/200
45000/45000 [==============================] - 36s 807us/step - loss: 0.9027 - acc: 0.8664 - val_loss: 1.0946 - val_acc: 0.8272
Epoch 2/200
45000/45000 [==============================] - 25s 555us/step - loss: 0.7966 - acc: 0.9064 - val_loss: 1.1261 - val_acc: 0.8316
Epoch 3/200
45000/45000 [==============================] - 25s 550us/step - loss: 0.7822 - acc: 0.9151 - val_loss: 1.1433 - val_acc: 0.8294
Epoch 4/200
45000/45000 [==============================] - 25s 551us/step - loss: 0.7842 - acc: 0.9186 - val_loss: 1.1284 - val_acc: 0.8410
Epoch 5/200
45000/45000 [==============================] - 25s 554us/step - loss: 0.7797 - acc: 0.9245 - val_loss: 1.1239 - val_acc: 0.8410
Epoch 6/200
45000/45000 [==============================] - 25s 553us/step - loss: 0.7767 - acc: 0.9280 - val_loss: 1.1483 - val_acc: 0.8440
Epoch 7/200
45000/45000 [==============================] - 25s 552us/step - loss: 0.7737 - acc: 0.9312 - val_lo

45000/45000 [==============================] - 25s 559us/step - loss: 0.7531 - acc: 0.9467 - val_loss: 1.2025 - val_acc: 0.8386
Epoch 117/200
45000/45000 [==============================] - 25s 557us/step - loss: 0.7550 - acc: 0.9457 - val_loss: 1.1680 - val_acc: 0.8546
Epoch 118/200
45000/45000 [==============================] - 25s 558us/step - loss: 0.7469 - acc: 0.9488 - val_loss: 1.2922 - val_acc: 0.8232
Epoch 119/200
45000/45000 [==============================] - 25s 554us/step - loss: 0.7501 - acc: 0.9476 - val_loss: 1.2306 - val_acc: 0.8400
Epoch 120/200
45000/45000 [==============================] - 25s 557us/step - loss: 0.7498 - acc: 0.9481 - val_loss: 1.2009 - val_acc: 0.8424
Epoch 121/200
45000/45000 [==============================] - 25s 559us/step - loss: 0.7494 - acc: 0.9469 - val_loss: 1.3484 - val_acc: 0.8154
Epoch 122/200
45000/45000 [==============================] - 25s 556us/step - loss: 0.7434 - acc: 0.9489 - val_loss: 1.2231 - val_acc: 0.8468
Epoch 123/200
45000/

In [64]:
import functools
import keras
top1_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=1) #top-1精度

top1_acc.__name__ = 'top1_acc'

model_9.compile(loss='categorical_crossentropy',
             optimizer='adam',metrics=['accuracy','top_k_categorical_accuracy',top1_acc])

model_9.evaluate(x_test,y_test,verbose=0)


[1.2642165331840516, 0.8276, 0.9851, 0.8276]

In [65]:
score=model_9.evaluate(x_test,y_test,verbose=0)
print("测试损失为：{:.4f}".format(score[0]))
print("测试准确率为：{:.4f}".format(score[1]))

测试损失为：1.2642
测试准确率为：0.8276


In [66]:
tensor =[]
for i in range(0,512):
   
    u=model_9.get_layer('conv2d_10').get_weights()[0][:,:,:,i].squeeze()
    v=np.mean(u, axis=1)
    #print(v)
    vT=v.T
    D=np.cov(vT)
    try:                 
        invD=np.linalg.inv(D)
        #t=np.mean(v,axis=1)
        a=[]

        for j in range(0,2): #表示filter的大小
            for k in range(j+1,3):
               #if (j!=k) and (j<k):
                tp=v[j]-v[k]
                d=np.sqrt(abs(np.dot(np.dot(tp,invD),tp.T)))
                a.append(d)
            
        print(i,a)
        tensor.append(a)
    except:
        print("不可逆")
print(tensor)

0 [22.949229388295407, 53.534523244216864, 39.17933075610269]
1 [23.404767660508856, 28.941987976991193, 16.727260762107193]
2 [22.57594843210242, 22.549156025996655, 19.898852256762815]
3 [36.490127038167415, 36.006299183008565, 41.52862618613472]
4 [32.836801753990095, 18.846180887143007, 21.722161948512564]
5 [16.745224741728638, 56.60837007578763, 50.52354928283453]
6 [42.43005032666072, 68.3105614501414, 69.15631663392068]
7 [17.566624710254885, 42.43754269762096, 14.699309771253686]
8 [18.83429623845748, 4.848284364839652, 7.488363903307869]
9 [15.665471237648589, 13.510809209096957, 17.771178680081473]
10 [15.176463414476215, 4.521213452521301, 5.142972369644025]
11 [61.37940730102402, 26.50682095248006, 73.81157711527716]
12 [35.53325699075749, 51.93071009279169, 8.612518235230208]
13 [12.325706873309855, 11.47908095219914, 9.78846447846448]
14 [7.232334714359688, 29.530072805364647, 22.419046214230942]
15 [13.064773763222687, 7.610253253988579, 42.899360075759496]
16 [11.78551

151 [10.160805769079198, 33.79671785456524, 6.577563989053267]
152 [21.155940449420783, 27.512384972338364, 5.35569792743364]
153 [41.77958220503521, 44.56054999965746, 39.28994836754828]
154 [11.381031260936263, 20.22063923636548, 14.670660876656557]
155 [13.990811670628796, 16.902109545184228, 12.456753842221719]
156 [14.09246627476047, 6.817956401682197, 19.190803318036927]
157 [12.080165892939801, 24.162099729558705, 10.68591593553197]
158 [6.843415683262314, 15.746591456503099, 5.567805128502129]
159 [19.193689075051555, 37.67028531834576, 12.524467440188399]
160 [9.020513838703735, 15.11653008818131, 18.048034260825517]
161 [9.213954932801258, 9.613512640148116, 11.745905040852715]
162 [20.980450579220424, 20.012862122766165, 5.637876776307755]
163 [8.071778377555738, 19.71232301950682, 13.139399340935961]
164 [33.72401422483705, 49.03103927460045, 39.44432112864051]
165 [45.37329249676587, 21.29587485972952, 49.90441490817186]
166 [31.953733137168296, 18.21309038722553, 45.65485

285 [12.491974218669895, 12.145654547255003, 15.435104446799182]
286 [6.334509993301761, 17.450042664166954, 18.041880391230084]
287 [24.20547930975148, 20.43308961742648, 9.144981193961135]
288 [10.75605768478448, 24.571973463722568, 9.0220160351397]
289 [22.551025685330796, 7.710219886108221, 18.02397963853982]
290 [10.737042446845923, 49.04193640639873, 40.939405184376895]
291 [9.9372080634079, 14.789583401541847, 49.38331088849752]
292 [9.350895704113471, 5.640764313982498, 15.634273066198878]
293 [51.62519921363651, 87.35648701014657, 138.10560879156031]
294 [21.485952501123233, 22.677888289676638, 12.057508997541088]
295 [10.940719348682617, 27.275742087334578, 11.798083110867731]
296 [19.074942733458705, 7.774505380861576, 11.56906257178279]
297 [26.480587272643277, 9.44070430698312, 28.960567302205533]
298 [12.483326129036943, 29.072139658989784, 4.806348956891307]
299 [11.933436356176152, 11.059818546296682, 10.91709419801359]
300 [22.686166823968907, 22.657599878546648, 24.32

420 [6.317753484763258, 3.180814201318097, 11.829814795223736]
421 [20.573893454496737, 15.281628516254253, 16.377198777833]
422 [11.143946191383964, 18.016960220286222, 31.63670364758213]
423 [9.708084337918887, 20.190929251490537, 21.052533409319548]
424 [1.6033336243394574, 20.492168326156204, 11.475789500484218]
425 [4.993114498794443, 7.477059167451054, 15.379983544012227]
426 [21.132196047212698, 19.02519157202936, 17.904516232689502]
427 [46.14093060881531, 48.988984718804865, 36.907927730970364]
428 [3.788600932844581, 11.763819015117265, 9.646487526237559]
429 [22.019196014261542, 8.909507824868694, 10.639109392940975]
430 [30.99001969914458, 50.195306866716, 45.020189545756466]
431 [1979.2618336768887, 1855.2389670875286, 1615.7964560859762]
432 [23.564308299486317, 3.961720767252925, 16.015749794078395]
433 [8.08959741000918, 12.591165225287899, 10.136866291714366]
434 [29.220392289136125, 21.653967688931257, 21.695849204919952]
435 [36.6172295617541, 23.915222080036205, 14.

In [67]:
entt=[]
for i in range (0,512):
    
    data=tensor[i]
    data0=np.array(data)
    print('四舍五入，精确到个位\n',np.round(data0,decimals=-2))
    data1=np.round(data0,decimals=-1)

    data1_value_list=set([data1[i] for i in range (data1.shape[0])])
    print(data1_value_list)
    ent=0.0
    for data1_value in data1_value_list:
        p=float(data1[data1==data1_value].shape[0])/data1.shape[0]
        print(p)
        logp=np.log2(p)
        ent-=p*logp
        print(ent)   
    entt.append(ent)
print(entt)

e=np.mean(entt)
print(e)
bb=[k for k in range(len(entt)) if entt[k]<np.mean(entt)]
print(bb)
c=len(bb)
print(c)

四舍五入，精确到个位
 [  0. 100.   0.]
{40.0, 50.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{20.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{20.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{40.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{20.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [  0. 100. 100.]
{50.0, 20.0, 60.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [  0. 100. 100.]
{40.0, 70.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.056

0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{20.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 50.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [100.   0.   0.]
{50.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500

 [0. 0. 0.]
{10.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [100.   0.   0.]
{50.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [100. 100.   0.]
{130.0, 60.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.333333333

0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{20.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{30.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到

0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
[1.584962500721156, 0.9182958340544896, 0.0, 0.0, 0.9182958340544896, 1.584962500721156, 0.9182958340544896, 1.584962500721156, 1.584962500721156, 0.9182958340544896, 1.584962500721156, 1.584962500721156, 1.584962500721156, 0.0, 1.584962500721156, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 1.584962500721156, 0.9182958340544896, 1.584962500721156, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.0, 0.0, 0.0, 1.584962500721156, 1.584962500721156, 0.0, 1.584962500721156, 0.9182958340544896, 1.584962500721156, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 1.584962500721156, 1.584962500721156, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 1.584962500721

In [68]:
from kerassurgeon.operations import delete_layer,insert_layer,delete_channels
model_10 = delete_channels(model_9, model_9.layers[37],bb)
model_10.summary()
model_10.save('ent_s_cifar10vgg16_10.h5')
model_10.save_weights('ent_s_cifar10vgg16_10_weights.h5',overwrite=True)

Deleting 338/512 channels from layer: conv2d_10
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 35)        980       
_________________________________________________________________
activation_1 (Activation)    multiple                  0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 35)        140       
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 18)        5688      
_________________________________________________________________
activation_2 (Activation)   

In [69]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model_10.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])
 
train_history=model_10.fit(x_train,y_train,epochs=200, batch_size=128,
             validation_split=0.1, verbose=1)

model_10.save('ent_s_cifar10vgg16_10.h5')
model_10.save_weights('ent_s_cifar10vgg16_10_weights.h5',overwrite=True)

Train on 45000 samples, validate on 5000 samples
Epoch 1/200
45000/45000 [==============================] - 36s 793us/step - loss: 0.8619 - acc: 0.8611 - val_loss: 1.0213 - val_acc: 0.8198
Epoch 2/200
45000/45000 [==============================] - 23s 519us/step - loss: 0.7823 - acc: 0.8880 - val_loss: 0.9550 - val_acc: 0.8474
Epoch 3/200
45000/45000 [==============================] - 23s 522us/step - loss: 0.7720 - acc: 0.8930 - val_loss: 0.9941 - val_acc: 0.8394
Epoch 4/200
45000/45000 [==============================] - 24s 523us/step - loss: 0.7615 - acc: 0.9004 - val_loss: 0.9645 - val_acc: 0.8482
Epoch 5/200
45000/45000 [==============================] - 23s 521us/step - loss: 0.7438 - acc: 0.9074 - val_loss: 0.9730 - val_acc: 0.8502
Epoch 6/200
45000/45000 [==============================] - 24s 523us/step - loss: 0.7369 - acc: 0.9105 - val_loss: 1.0152 - val_acc: 0.8378
Epoch 7/200
45000/45000 [==============================] - 24s 523us/step - loss: 0.7360 - acc: 0.9124 - val_lo

45000/45000 [==============================] - 24s 524us/step - loss: 0.7108 - acc: 0.9347 - val_loss: 1.1114 - val_acc: 0.8464
Epoch 117/200
45000/45000 [==============================] - 24s 527us/step - loss: 0.7115 - acc: 0.9352 - val_loss: 1.1021 - val_acc: 0.8400
Epoch 118/200
45000/45000 [==============================] - 24s 531us/step - loss: 0.7081 - acc: 0.9361 - val_loss: 1.0455 - val_acc: 0.8554
Epoch 119/200
45000/45000 [==============================] - 24s 527us/step - loss: 0.7075 - acc: 0.9359 - val_loss: 1.0861 - val_acc: 0.8450
Epoch 120/200
45000/45000 [==============================] - 24s 525us/step - loss: 0.7112 - acc: 0.9340 - val_loss: 1.1056 - val_acc: 0.8452
Epoch 121/200
45000/45000 [==============================] - 24s 526us/step - loss: 0.7022 - acc: 0.9379 - val_loss: 1.0550 - val_acc: 0.8560
Epoch 122/200
45000/45000 [==============================] - 24s 525us/step - loss: 0.7155 - acc: 0.9332 - val_loss: 1.1743 - val_acc: 0.8296
Epoch 123/200
45000/

Epoch 174/200
45000/45000 [==============================] - 24s 524us/step - loss: 0.6956 - acc: 0.9398 - val_loss: 1.0784 - val_acc: 0.8430
Epoch 175/200
45000/45000 [==============================] - 24s 528us/step - loss: 0.7060 - acc: 0.9347 - val_loss: 1.1085 - val_acc: 0.8404
Epoch 176/200
45000/45000 [==============================] - 24s 529us/step - loss: 0.7021 - acc: 0.9364 - val_loss: 1.0980 - val_acc: 0.8466
Epoch 177/200
45000/45000 [==============================] - 24s 525us/step - loss: 0.7054 - acc: 0.9355 - val_loss: 1.0687 - val_acc: 0.8436
Epoch 178/200
45000/45000 [==============================] - 24s 525us/step - loss: 0.7050 - acc: 0.9356 - val_loss: 1.0877 - val_acc: 0.8422
Epoch 179/200
45000/45000 [==============================] - 24s 528us/step - loss: 0.7003 - acc: 0.9376 - val_loss: 1.1832 - val_acc: 0.8268
Epoch 180/200
45000/45000 [==============================] - 23s 522us/step - loss: 0.7048 - acc: 0.9361 - val_loss: 1.0755 - val_acc: 0.8488
Epoch 

In [70]:
import functools
import keras
top1_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=1) #top-1精度

top1_acc.__name__ = 'top1_acc'

model_10.compile(loss='categorical_crossentropy',
             optimizer='adam',metrics=['accuracy','top_k_categorical_accuracy',top1_acc])

model_10.evaluate(x_test,y_test,verbose=0)

[1.0670612928390504, 0.8477, 0.9903, 0.8477]

In [71]:
score=model_10.evaluate(x_test,y_test,verbose=0)
print("测试损失为：{:.4f}".format(score[0]))
print("测试准确率为：{:.4f}".format(score[1]))

测试损失为：1.0671
测试准确率为：0.8477


In [72]:
tensor =[]
for i in range(0,512):
   
    u=model_10.get_layer('conv2d_11').get_weights()[0][:,:,:,i].squeeze()
    v=np.mean(u, axis=1)
    #print(v)
    vT=v.T
    D=np.cov(vT)
    try:                 
        invD=np.linalg.inv(D)
        #t=np.mean(v,axis=1)
        a=[]

        for j in range(0,2): #表示filter的大小
            for k in range(j+1,3):
               #if (j!=k) and (j<k):
                tp=v[j]-v[k]
                d=np.sqrt(abs(np.dot(np.dot(tp,invD),tp.T)))
                a.append(d)
            
        print(i,a)
        tensor.append(a)
    except:
        print("不可逆")
print(tensor)

0 [2.3281819381407804, 18.72770877746713, 15.709410302490731]
1 [10.742548023069485, 15.257152568493606, 16.240141386957553]
2 [8.03551272063785, 11.54460061764895, 12.347947577925938]
3 [26.01082825758357, 54.61298613608037, 14.263271643236344]
4 [14.465065670345645, 26.338738069914047, 6.213496764138842]
5 [15.31061453596898, 9.569112174606962, 14.199621947806785]
6 [17.177963343791824, 24.40858610453622, 23.09388090723744]
7 [13.151921155874225, 22.069745395092333, 78.77769637122324]
8 [5.798164584111184, 20.378564745085487, 9.562649295613811]
9 [7.172817891290523, 26.154100117809676, 14.263863604167321]
10 [8.80283214987925, 1.613205138990416, 10.305568816446945]
11 [249.88318865260106, 86.80505523727345, 116.58460748797415]
12 [24.077165786282027, 16.272449765604755, 13.726566460163237]
13 [15.766893946351637, 5.018217088427127, 16.66185146709463]
14 [25.525684081502583, 7.619157522691785, 13.089894876633545]
15 [14.41488024451811, 8.333538940197949, 18.489448244559508]
16 [11.127

132 [7.695371737954102, 9.23161550946262, 10.21003336212581]
133 [30.72600410006406, 10.272260173449897, 4.547579943356361]
134 [13.046458236525153, 10.617387793576574, 12.762522886668577]
135 [17.779417898928457, 20.34990539247608, 20.538530542356103]
136 [48.376162492845374, 99.49053519007336, 172.80793435548176]
137 [22.33774997206281, 11.783170750929337, 20.139981445258496]
138 [21.477593445413092, 14.604191400050807, 15.251748391280794]
139 [13.431044407367606, 9.642039165922899, 9.390342378092424]
140 [7.6024228243311684, 19.762867550505344, 10.692443685822587]
141 [1.5184664489034034, 8.748116808281953, 19.497988377028346]
142 [26.54803785771264, 26.333304364957048, 6.811432431067674]
143 [24.56441420456262, 36.30550659910593, 12.387142947658722]
144 [9.833745225803627, 23.989253976217128, 13.104789068161153]
145 [32.580529696407055, 17.809572561533415, 18.0650514694735]
146 [19.622300369310608, 25.461627886466847, 7.558867788152504]
147 [8.635689137895586, 15.706348464116777, 7

275 [1.6715756442440561, 13.07731769945907, 6.315347501386043]
276 [8.556982671383324, 4.119960249366475, 12.5991845911575]
277 [3.4617387175710514, 15.181055106509596, 19.928034242420647]
278 [51.55020919890556, 30.87475175796358, 19.595033059098373]
279 [6.373156823376875, 4.652809984497399, 13.01737151042778]
280 [14.96452447107036, 7.2341228218699545, 7.159839179827551]
281 [28.81447003760391, 31.304994700147603, 25.07573121907292]
282 [9.071054774887202, 25.044856118017194, 48.35444312761005]
283 [16.42854369676512, 9.490260464698554, 22.334269218146282]
284 [21.164247266287468, 20.08971952978126, 8.287065529690459]
285 [26.335851134478585, 11.801137798217574, 3.5513586533764947]
286 [64.05977283162389, 71.74238657682278, 35.98612030341959]
287 [34.75100835187947, 30.312835345428795, 26.845292656550736]
288 [12.502148598547912, 11.655859503456309, 15.712796382491033]
289 [32.46297623054037, 53.04704686896863, 47.38084210531086]
290 [17.158164405670508, 20.027972591520815, 11.53160

419 [5.117137926167314, 24.06027461078351, 14.929503898350019]
420 [7.546191174661889, 12.9452838793791, 11.634186222287772]
421 [14.137979569804985, 6.632439808433998, 13.702050892263347]
422 [8.115358882825706, 12.78868934865288, 74.05123970287049]
423 [60.45810298582223, 28.905491009619645, 22.40275698665635]
424 [23.224680414043863, 18.038451080732177, 14.743613331328271]
425 [11.924422700571514, 9.507924489072511, 14.791657749585253]
426 [19.800981681262346, 19.844945436048476, 23.67612661321588]
427 [14.199386373210002, 8.242807200489018, 39.369668278671334]
428 [14.037278067786794, 13.72751523371518, 26.446151742620575]
429 [3.585119066775021, 14.308490816329801, 17.54673739181462]
430 [74.09709674125834, 435.11659098902015, 125.71543187696457]
431 [79.85472477625727, 18.322861731184652, 37.13980421315623]
432 [15.372455525938012, 17.463858690767996, 16.97081153247587]
433 [14.040278633102472, 20.46662460823998, 6.042914776847688]
434 [24.862331557434466, 23.298223158412153, 9.6

In [73]:
entt=[]
for i in range (0,512):
    
    data=tensor[i]
    data0=np.array(data)
    print('四舍五入，精确到个位\n',np.round(data0,decimals=-2))
    data1=np.round(data0,decimals=-1)

    data1_value_list=set([data1[i] for i in range (data1.shape[0])])
    print(data1_value_list)
    ent=0.0
    for data1_value in data1_value_list:
        p=float(data1[data1==data1_value].shape[0])/data1.shape[0]
        print(p)
        logp=np.log2(p)
        ent-=p*logp
        print(ent)   
    entt.append(ent)
print(entt)

e=np.mean(entt)
print(e)
bb=[k for k in range(len(entt)) if entt[k]<np.mean(entt)]
print(bb)
c=len(bb)
print(c)

四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [  0. 100.   0.]
{50.0, 10.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{20.0}
1.0
0.0
四舍五入，精确到个位
 [  0.   0. 100.]
{80.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 30.0}
0.66

{20.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{20.0}
1.0
0.0
四舍五入，精确到个位
 [100.   0. 100.]
{50.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 30.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [100. 100.   0.]
{50.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{

0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [100. 200.   0.]
{40.0, 150.0, 70.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五

 [100.   0. 100.]
{80.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [100. 100. 100.]
{90.0, 100.0, 110.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [  0. 100.   0.]
{10.0, 60.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精

0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{20.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{20.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{20.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 40.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{50.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [  0.   0. 100.]
{40.0, 50.0, 60.0}
0

In [74]:
from kerassurgeon.operations import delete_layer,insert_layer,delete_channels
model_11 = delete_channels(model_10, model_10.layers[41],bb)
model_11.summary()
model_11.save('ent_s_cifar10vgg16_11.h5')
model_11.save_weights('ent_s_cifar10vgg16_11_weights.h5',overwrite=True)

Deleting 361/512 channels from layer: conv2d_11
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 35)        980       
_________________________________________________________________
activation_1 (Activation)    multiple                  0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 35)        140       
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 18)        5688      
_________________________________________________________________
activation_2 (Activation)   

In [4]:
from keras.models import load_model

model_11 = load_model('ent_s_cifar10vgg16_11.h5')
model_11.load_weights('ent_s_cifar10vgg16_11_weights.h5')

model_11.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 35)        980       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 35)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 35)        140       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 35)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 18)        5688      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 18)        0         
__________

C:\Users\dell\Anaconda3\envs\tensorflow\lib\site-packages\keras\models.py:282: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model_11.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])
 
train_history=model_11.fit(x_train,y_train,epochs=200, batch_size=128,
             validation_split=0.1, verbose=1)

model_11.save('ent_s_cifar10vgg16_11.h5')
model_11.save_weights('ent_s_cifar10vgg16_11_weights.h5',overwrite=True)

Train on 45000 samples, validate on 5000 samples
Epoch 1/200
45000/45000 [==============================] - 29s 637us/step - loss: 0.7336 - acc: 0.9072 - val_loss: 0.9988 - val_acc: 0.8456
Epoch 2/200
45000/45000 [==============================] - 19s 419us/step - loss: 0.7311 - acc: 0.9119 - val_loss: 0.9900 - val_acc: 0.8538
Epoch 3/200
45000/45000 [==============================] - 19s 414us/step - loss: 0.7146 - acc: 0.9202 - val_loss: 1.0118 - val_acc: 0.8460
Epoch 4/200
45000/45000 [==============================] - 18s 407us/step - loss: 0.7179 - acc: 0.9204 - val_loss: 1.0419 - val_acc: 0.8474
Epoch 5/200
45000/45000 [==============================] - 18s 400us/step - loss: 0.7282 - acc: 0.9187 - val_loss: 1.0146 - val_acc: 0.8496
Epoch 6/200
45000/45000 [==============================] - 18s 400us/step - loss: 0.7293 - acc: 0.9194 - val_loss: 1.0168 - val_acc: 0.8512
Epoch 7/200
45000/45000 [==============================] - 18s 398us/step - loss: 0.7221 - acc: 0.9223 - val_lo

45000/45000 [==============================] - 19s 427us/step - loss: 0.7177 - acc: 0.9255 - val_loss: 0.9885 - val_acc: 0.8626
Epoch 58/200
45000/45000 [==============================] - 19s 426us/step - loss: 0.7051 - acc: 0.9319 - val_loss: 1.0079 - val_acc: 0.8498
Epoch 59/200
45000/45000 [==============================] - 19s 428us/step - loss: 0.7060 - acc: 0.9288 - val_loss: 1.0156 - val_acc: 0.8556
Epoch 60/200
45000/45000 [==============================] - 19s 430us/step - loss: 0.7091 - acc: 0.9293 - val_loss: 1.0056 - val_acc: 0.8566
Epoch 61/200
45000/45000 [==============================] - 19s 423us/step - loss: 0.7037 - acc: 0.9314 - val_loss: 1.0299 - val_acc: 0.8488
Epoch 62/200
45000/45000 [==============================] - 19s 419us/step - loss: 0.7179 - acc: 0.9268 - val_loss: 0.9946 - val_acc: 0.8564
Epoch 63/200
45000/45000 [==============================] - 21s 473us/step - loss: 0.7126 - acc: 0.9284 - val_loss: 1.0640 - val_acc: 0.8488
Epoch 64/200
45000/45000 [

45000/45000 [==============================] - 21s 474us/step - loss: 0.6949 - acc: 0.9322 - val_loss: 1.0177 - val_acc: 0.8558loss: 0.6947 - acc: 0.93
Epoch 114/200
45000/45000 [==============================] - 22s 482us/step - loss: 0.7016 - acc: 0.9297 - val_loss: 0.9953 - val_acc: 0.8572
Epoch 115/200
45000/45000 [==============================] - 21s 476us/step - loss: 0.6968 - acc: 0.9324 - val_loss: 1.0537 - val_acc: 0.8466
Epoch 116/200
45000/45000 [==============================] - 22s 481us/step - loss: 0.6929 - acc: 0.9333 - val_loss: 1.0284 - val_acc: 0.8492
Epoch 117/200
45000/45000 [==============================] - 22s 479us/step - loss: 0.6947 - acc: 0.9318 - val_loss: 1.0184 - val_acc: 0.8568
Epoch 118/200
45000/45000 [==============================] - 21s 475us/step - loss: 0.6944 - acc: 0.9308 - val_loss: 1.0621 - val_acc: 0.8410
Epoch 119/200
45000/45000 [==============================] - 22s 479us/step - loss: 0.6951 - acc: 0.9306 - val_loss: 1.0978 - val_acc: 0.8

45000/45000 [==============================] - 21s 472us/step - loss: 0.6972 - acc: 0.9313 - val_loss: 1.0733 - val_acc: 0.8450
Epoch 170/200
45000/45000 [==============================] - 22s 479us/step - loss: 0.6935 - acc: 0.9327 - val_loss: 1.0785 - val_acc: 0.8406
Epoch 171/200
45000/45000 [==============================] - 22s 483us/step - loss: 0.6899 - acc: 0.9343 - val_loss: 1.0078 - val_acc: 0.8588
Epoch 172/200
45000/45000 [==============================] - 21s 468us/step - loss: 0.6841 - acc: 0.9338 - val_loss: 1.0189 - val_acc: 0.8540
Epoch 173/200
45000/45000 [==============================] - 22s 481us/step - loss: 0.6936 - acc: 0.9318 - val_loss: 1.1107 - val_acc: 0.8334
Epoch 174/200
45000/45000 [==============================] - 22s 481us/step - loss: 0.6868 - acc: 0.9330 - val_loss: 1.0076 - val_acc: 0.8550
Epoch 175/200
45000/45000 [==============================] - 21s 470us/step - loss: 0.6931 - acc: 0.9318 - val_loss: 1.0365 - val_acc: 0.8472
Epoch 176/200
45000/

In [6]:
import functools
import keras
top1_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=1) #top-1精度

top1_acc.__name__ = 'top1_acc'

model_11.compile(loss='categorical_crossentropy',
             optimizer='adam',metrics=['accuracy','top_k_categorical_accuracy',top1_acc])

model_11.evaluate(x_test,y_test,verbose=0)

[1.0988508651733397, 0.8347, 0.9861, 0.8347]

In [7]:
score=model_11.evaluate(x_test,y_test,verbose=0)
print("测试损失为：{:.4f}".format(score[0]))
print("测试准确率为：{:.4f}".format(score[1]))

测试损失为：1.0989
测试准确率为：0.8347


In [8]:
tensor =[]
for i in range(0,512):
   
    u=model_11.get_layer('conv2d_12').get_weights()[0][:,:,:,i].squeeze()
    v=np.mean(u, axis=1)
    #print(v)
    vT=v.T
    D=np.cov(vT)
    try:                 
        invD=np.linalg.inv(D)
        #t=np.mean(v,axis=1)
        a=[]

        for j in range(0,2): #表示filter的大小
            for k in range(j+1,3):
               #if (j!=k) and (j<k):
                tp=v[j]-v[k]
                d=np.sqrt(abs(np.dot(np.dot(tp,invD),tp.T)))
                a.append(d)
            
        print(i,a)
        tensor.append(a)
    except:
        print("不可逆")
print(tensor)

0 [17.907445418314477, 9.669140258093709, 9.349718719984162]
1 [23.553164061819764, 12.610825706896733, 35.37400042735825]
2 [37.315464304581276, 22.58337582048124, 37.38159039930284]
3 [11.150724854023004, 6.567108815656868, 21.35252318469996]
4 [13.936463099342308, 17.70229521487634, 19.052442774259358]
5 [9.204040119158309, 12.095950407004114, 41.32658020478375]
6 [18.519332063959943, 15.234491028955182, 14.601617126199379]
7 [43.88360334202945, 38.21251814183237, 57.672819507588095]
8 [18.302974736393256, 3.4598704602682973, 9.587874461673456]
9 [6.808420447170042, 8.328824909386599, 12.445422950920225]
10 [74.88970912641531, 24.869886994237433, 54.468612311797074]
11 [10.035937821187646, 19.368263070114985, 25.769116018852227]
12 [1.9606937697719733, 14.737452431392501, 5.139408048690275]
13 [1.6918035661648303, 10.304959436028614, 7.054426321473466]
14 [12.559963976207024, 23.490443559163847, 16.46902552452011]
15 [20.139881579023495, 8.921384620885481, 4.222965705535409]
16 [25.

135 [15.469149174019245, 48.45756727396496, 45.67222024511092]
136 [34.75613275105854, 23.350517303160444, 17.091966386036635]
137 [7.490977455094032, 9.888719854326991, 9.722620104565069]
138 [14.972537220621843, 20.79468457776386, 19.19944167914827]
139 [34.834068670831215, 17.80965165236127, 11.992365541907146]
140 [75.66636305872747, 81.80395822251869, 54.74742754462447]
141 [10.294041615694223, 8.20862722516189, 9.567120202597398]
142 [3.7279379283721807, 17.460918711539108, 22.7380040054707]
143 [24.52865940677764, 7.102917834582598, 4.461558070000581]
144 [103.79063634515461, 52.67290750881742, 48.19884740314037]
145 [33.204756192698184, 25.73285274485096, 45.77748316348022]
146 [24.808527323364068, 17.194441063102893, 41.30482058344008]
147 [73.13432839388096, 25.912954201515294, 36.72212506305604]
148 [7.984066369349418, 14.988487355572623, 10.017910082205825]
149 [1.034543874732995, 11.20794859063926, 3.369894899715989]
150 [13.040175237882357, 26.085673524812485, 10.88794166

274 [35.20425953668629, 28.782535884599227, 34.37323925710766]
275 [12.800759882089883, 7.554791644067911, 6.319937941270662]
276 [13.496215086638557, 17.101146820160842, 7.968721860948135]
277 [65.50521550271124, 253.0518516510469, 138.7144270101066]
278 [17.20425569316175, 40.87434694660904, 4.8823874815023895]
279 [7.161795054524748, 12.661638459724085, 9.459550915012702]
280 [19.248152331446075, 7.332199395537579, 8.980111956716211]
281 [61.26283805363165, 24.875898709188643, 45.23854448936746]
282 [14.392912808120977, 7.461847155974583, 11.766700599209935]
283 [12.494224720991358, 29.55565526972548, 30.434324752965118]
284 [21.99092543454903, 12.903761524511983, 17.16323613021055]
285 [16.604398798386878, 11.255458365926945, 19.002726566282433]
286 [37.85126769085883, 18.727875219111905, 28.76358838929539]
287 [16.619523544643275, 6.703412625559768, 14.617460964953889]
288 [28.60676476130235, 58.065793834014, 47.596270208327674]
289 [17.756388166159276, 10.587423652409681, 13.1753

415 [13.188973104993988, 16.1632552168874, 20.28375962163888]
416 [7.60278724456422, 10.39244391790752, 11.89015252957952]
417 [10.215126129973754, 8.07888011243853, 11.7357537872924]
418 [24.946253108000366, 16.757974633630734, 17.398386303843907]
419 [15.55658717604254, 5.462580225025876, 16.223122854340296]
420 [16.4366948615901, 21.370561261551554, 16.810325737224773]
421 [32.994325655702504, 18.422745804593077, 45.38266755046605]
422 [16.79384755374924, 8.027410041925178, 7.3298600670336755]
423 [15.261222118521252, 16.39263180792454, 12.742748666798898]
424 [35.86753400678027, 8.015660574017977, 5.599578633804482]
425 [15.38400902460997, 9.442944970648947, 5.318947324219087]
426 [18.909305054549904, 18.250783302532685, 27.634708286662413]
427 [13.829372164417844, 13.40554891322569, 13.212466304306616]
428 [13.939626830810464, 17.746062742017678, 17.169188326589126]
429 [3.7553684590238285, 18.979845349613342, 13.077202749606009]
430 [23.901524075391634, 8.631015007029863, 15.7609

In [9]:
entt=[]
for i in range (0,512):
    
    data=tensor[i]
    data0=np.array(data)
    print('四舍五入，精确到个位\n',np.round(data0,decimals=-2))
    data1=np.round(data0,decimals=-1)

    data1_value_list=set([data1[i] for i in range (data1.shape[0])])
    print(data1_value_list)
    ent=0.0
    for data1_value in data1_value_list:
        p=float(data1[data1==data1_value].shape[0])/data1.shape[0]
        print(p)
        logp=np.log2(p)
        ent-=p*logp
        print(ent)   
    entt.append(ent)
print(entt)

e=np.mean(entt)
print(e)
bb=[k for k in range(len(entt)) if entt[k]<np.mean(entt)]
print(bb)
c=len(bb)
print(c)

四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [  0.   0. 100.]
{40.0, 60.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.33333333

1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{20.0, 30.0}
0.

 [0. 0. 0.]
{20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{50.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [  0.   0. 100.]
{10.0, 60.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.

 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896


In [10]:
from kerassurgeon.operations import delete_layer,insert_layer,delete_channels
model_12 = delete_channels(model_11, model_11.layers[45],bb)
model_12.summary()
model_12.save('ent_s_cifar10vgg16_12.h5')
model_12.save_weights('ent_s_cifar10vgg16_12_weights.h5',overwrite=True)

Deleting 367/512 channels from layer: conv2d_12
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 35)        980       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 35)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 35)        140       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 35)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 18)        5688      
_________________________________________________________________
activation_2 (Activation)   

In [11]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model_12.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])
 
train_history=model_12.fit(x_train,y_train,epochs=200, batch_size=128,
             validation_split=0.1, verbose=1)

model_12.save('ent_s_cifar10vgg16_12.h5')
model_12.save_weights('ent_s_cifar10vgg16_12_weights.h5',overwrite=True)

Train on 45000 samples, validate on 5000 samples
Epoch 1/200
45000/45000 [==============================] - 25s 566us/step - loss: 0.7001 - acc: 0.9244 - val_loss: 1.0621 - val_acc: 0.8482
Epoch 2/200
45000/45000 [==============================] - 20s 440us/step - loss: 0.7145 - acc: 0.9232 - val_loss: 1.0109 - val_acc: 0.8584
Epoch 3/200
45000/45000 [==============================] - 20s 440us/step - loss: 0.7141 - acc: 0.9270 - val_loss: 1.0496 - val_acc: 0.8472
Epoch 4/200
45000/45000 [==============================] - 20s 433us/step - loss: 0.7176 - acc: 0.9253 - val_loss: 1.1444 - val_acc: 0.8284
Epoch 5/200
45000/45000 [==============================] - 20s 441us/step - loss: 0.7211 - acc: 0.9255 - val_loss: 1.1140 - val_acc: 0.8372
Epoch 6/200
45000/45000 [==============================] - 20s 442us/step - loss: 0.7278 - acc: 0.9239 - val_loss: 1.2477 - val_acc: 0.8054
Epoch 7/200
45000/45000 [==============================] - 19s 432us/step - loss: 0.7300 - acc: 0.9249 - val_lo

Epoch 59/200
45000/45000 [==============================] - 19s 432us/step - loss: 0.7077 - acc: 0.9304 - val_loss: 1.0270 - val_acc: 0.8524
Epoch 60/200
45000/45000 [==============================] - 20s 441us/step - loss: 0.7011 - acc: 0.9319 - val_loss: 1.0891 - val_acc: 0.8440
Epoch 61/200
45000/45000 [==============================] - 20s 440us/step - loss: 0.7058 - acc: 0.9308 - val_loss: 1.0553 - val_acc: 0.8452
Epoch 62/200
45000/45000 [==============================] - 19s 432us/step - loss: 0.7071 - acc: 0.9313 - val_loss: 1.1366 - val_acc: 0.8306
Epoch 63/200
45000/45000 [==============================] - 20s 441us/step - loss: 0.7086 - acc: 0.9323 - val_loss: 1.0229 - val_acc: 0.8536
Epoch 64/200
45000/45000 [==============================] - 20s 441us/step - loss: 0.7022 - acc: 0.9318 - val_loss: 1.1866 - val_acc: 0.8254
Epoch 65/200
45000/45000 [==============================] - 20s 434us/step - loss: 0.7047 - acc: 0.9324 - val_loss: 1.0782 - val_acc: 0.8494
Epoch 66/200


Epoch 117/200
45000/45000 [==============================] - 20s 437us/step - loss: 0.6965 - acc: 0.9319 - val_loss: 1.0019 - val_acc: 0.8610
Epoch 118/200
45000/45000 [==============================] - 20s 442us/step - loss: 0.6863 - acc: 0.9343 - val_loss: 1.0282 - val_acc: 0.8504
Epoch 119/200
45000/45000 [==============================] - 20s 440us/step - loss: 0.7015 - acc: 0.9299 - val_loss: 1.0739 - val_acc: 0.8390
Epoch 120/200
45000/45000 [==============================] - 20s 437us/step - loss: 0.6982 - acc: 0.9319 - val_loss: 1.0102 - val_acc: 0.8566
Epoch 121/200
45000/45000 [==============================] - 20s 441us/step - loss: 0.6942 - acc: 0.9324 - val_loss: 1.0793 - val_acc: 0.8416
Epoch 122/200
45000/45000 [==============================] - 20s 439us/step - loss: 0.6928 - acc: 0.9333 - val_loss: 1.1466 - val_acc: 0.8280
Epoch 123/200
45000/45000 [==============================] - 19s 431us/step - loss: 0.6917 - acc: 0.9327 - val_loss: 1.0446 - val_acc: 0.8470
Epoch 

Epoch 175/200
45000/45000 [==============================] - 20s 439us/step - loss: 0.6781 - acc: 0.9358 - val_loss: 1.0416 - val_acc: 0.8474
Epoch 176/200
45000/45000 [==============================] - 20s 442us/step - loss: 0.6828 - acc: 0.9337 - val_loss: 1.0154 - val_acc: 0.8610
Epoch 177/200
45000/45000 [==============================] - 20s 436us/step - loss: 0.6778 - acc: 0.9341 - val_loss: 1.0308 - val_acc: 0.8572
Epoch 178/200
45000/45000 [==============================] - 20s 438us/step - loss: 0.6888 - acc: 0.9319 - val_loss: 1.0487 - val_acc: 0.8432
Epoch 179/200
45000/45000 [==============================] - 20s 440us/step - loss: 0.6850 - acc: 0.9340 - val_loss: 1.0133 - val_acc: 0.8492
Epoch 180/200
45000/45000 [==============================] - 20s 434us/step - loss: 0.6804 - acc: 0.9356 - val_loss: 1.0481 - val_acc: 0.8520
Epoch 181/200
45000/45000 [==============================] - 20s 440us/step - loss: 0.6807 - acc: 0.9350 - val_loss: 1.0363 - val_acc: 0.8478
Epoch 

In [12]:
import functools
import keras
top1_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=1) #top-1精度

top1_acc.__name__ = 'top1_acc'

model_12.compile(loss='categorical_crossentropy',
             optimizer='adam',metrics=['accuracy','top_k_categorical_accuracy',top1_acc])

model_12.evaluate(x_test,y_test,verbose=0)

[1.0741897413253785, 0.8419, 0.9883, 0.8419]

In [13]:
score=model_12.evaluate(x_test,y_test,verbose=0)
print("测试损失为：{:.4f}".format(score[0]))
print("测试准确率为：{:.4f}".format(score[1]))

测试损失为：1.0742
测试准确率为：0.8419


In [14]:
tensor =[]
for i in range(0,512):
   
    u=model_12.get_layer('conv2d_13').get_weights()[0][:,:,:,i].squeeze()
    v=np.mean(u, axis=1)
    #print(v)
    vT=v.T
    D=np.cov(vT)
    try:                 
        invD=np.linalg.inv(D)
        #t=np.mean(v,axis=1)
        a=[]

        for j in range(0,2): #表示filter的大小
            for k in range(j+1,3):
               #if (j!=k) and (j<k):
                tp=v[j]-v[k]
                d=np.sqrt(abs(np.dot(np.dot(tp,invD),tp.T)))
                a.append(d)
            
        print(i,a)
        tensor.append(a)
    except:
        print("不可逆")
print(tensor)

0 [40.72542949939456, 47.152696560299866, 146.37787336726626]
1 [24.816910589525364, 9.985598159393913, 5.404286711655336]
2 [5.73543615683133, 11.979658857544939, 9.770947963364824]
3 [36.85660515727788, 46.597549948729764, 32.21960083275572]
4 [10.376855431528888, 11.157746714147105, 16.68262041736454]
5 [47.56437272383926, 57.282058801679625, 39.77433821093442]
6 [7.4525248607795405, 12.028932293673375, 12.484575240160972]
7 [6.065565690895922, 2.833006529099319, 5.567035626083136]
8 [22.764189476566575, 22.10295705215071, 17.109720534924538]
9 [142.38559593044093, 150.12528825235128, 96.75203464152777]
10 [19.413047693390382, 16.630037103956788, 6.550753148109062]
11 [8.919027324981524, 20.85240741617257, 9.672806258964455]
12 [34.49736441344467, 57.5504533420077, 44.055624610768724]
13 [26.530608134598765, 49.30466297374509, 34.289082247071526]
14 [3.290704747580416, 5.91399654618731, 5.61571059886161]
15 [10.319952855279714, 10.012247929632544, 7.599589155410931]
16 [25.776429288

134 [29.96733890270245, 30.93048849318867, 38.53759787587603]
135 [23.14077785526459, 15.209406465615807, 11.308383066062856]
136 [53.878202498060354, 30.0474985527129, 22.68859532762383]
137 [9.989395598608887, 7.094562568003121, 6.500481440981012]
138 [18.774319529996077, 27.194874186546528, 18.342356263328412]
139 [14.708483442165512, 27.94058162752827, 20.860997123367333]
140 [4.897906601653581, 28.388577897080644, 12.168992314524578]
141 [18.492339507083777, 19.086408012046352, 31.53245377298498]
142 [11.27831505403653, 4.933982851899358, 21.558929943926913]
143 [27.989506730738025, 30.331800581035935, 18.300013767299788]
144 [11.101000777656646, 20.963714131342854, 26.006783260790453]
145 [9.497953570953072, 18.733605757980353, 9.897302130397602]
146 [20.721148989239595, 11.346614192317592, 6.3064750585859]
147 [35.97841868960362, 68.9199548564969, 34.685776277620484]
148 [7.021154352955685, 12.373674808215547, 12.1964669521217]
149 [18.55753092933204, 49.322836485182926, 11.4914

279 [17.85177036556701, 12.722651152804476, 13.391567415171497]
280 [0.6793148729376078, 12.42981367448776, 25.032968945481954]
281 [11.56561214731817, 9.362873830831553, 11.20289403020251]
282 [11.598362871269382, 25.81366526672871, 5.327981628099905]
283 [122.03149678966072, 109.37298351935812, 13.458565882198176]
284 [10.70874142271188, 14.779337271313404, 6.77860542448478]
285 [13.275495085670762, 11.153096536058662, 5.4590583700716575]
286 [9.283380610661228, 8.939096748327783, 5.597045141005682]
287 [11.218593840720928, 24.04032558907769, 24.827656380784408]
288 [68.11959750250884, 60.8464238970832, 17.35547852413942]
289 [18.969354811552044, 8.835402319879329, 7.242087025032868]
290 [10.702574783995983, 11.465842717854434, 3.4729704055030473]
291 [40.55176136657589, 58.34016571722338, 58.823380147968976]
292 [14.699588297573646, 6.812809053462756, 13.783354825545455]
293 [2.3498559380060566, 25.010802872257848, 13.325368544758442]
294 [14.195814435584445, 13.209920021727545, 8.4

423 [11.969331274035886, 12.192167337184115, 8.42194458432017]
424 [23.916155589244383, 23.836334155197857, 10.805194227758047]
425 [16.645995513673412, 8.309908039718042, 23.98980885933996]
426 [12.405407211542181, 14.004271111197701, 6.332781474849019]
427 [14.280221355296039, 7.30989230357104, 22.372374993321493]
428 [18.42134116760668, 8.232418428648819, 7.288536289924215]
429 [14.604044581332424, 14.905140001726563, 14.572714272712167]
430 [18.224099121538394, 11.712193844762286, 15.750618778668171]
431 [44.287320880489986, 37.52039001530547, 20.140079156479505]
432 [29.66737039852918, 46.676537039835424, 28.37048396332737]
433 [17.168426078806892, 11.746331575635507, 15.994308357137196]
434 [49.387758330507346, 8.013732032261867, 15.792630282805638]
435 [23.587356024354776, 6.081949006824922, 18.506195165159337]
436 [16.298739347678946, 6.928326891416715, 14.48205656489913]
437 [20.480237553157547, 6.452036176917104, 6.846178085676587]
438 [8.532299955265671, 14.957934563757474, 

In [15]:
entt=[]
for i in range (0,512):
    
    data=tensor[i]
    data0=np.array(data)
    print('四舍五入，精确到个位\n',np.round(data0,decimals=-2))
    data1=np.round(data0,decimals=-1)

    data1_value_list=set([data1[i] for i in range (data1.shape[0])])
    print(data1_value_list)
    ent=0.0
    for data1_value in data1_value_list:
        p=float(data1[data1==data1_value].shape[0])/data1.shape[0]
        print(p)
        logp=np.log2(p)
        ent-=p*logp
        print(ent)   
    entt.append(ent)
print(entt)

e=np.mean(entt)
print(e)
bb=[k for k in range(len(entt)) if entt[k]<np.mean(entt)]
print(bb)
c=len(bb)
print(c)

四舍五入，精确到个位
 [  0.   0. 100.]
{40.0, 50.0, 150.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 50.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [  0. 100.   0.]
{40.0, 50.0, 60.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{20.0}
1.0
0.0
四舍五入，精确到个位
 [100. 200. 100.]
{100.0, 140.0, 150.0}
0.3333333333333333
0

 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [100.   0. 100.]
{40.0, 100.0, 70.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.666666666666

0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [  0. 100.   0.]
{50.0, 10.0, 60.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 10.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [  0.   0. 100.]
{50.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.91

 [0. 0. 0.]
{20.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 10.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0

0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 

In [16]:
from kerassurgeon.operations import delete_layer,insert_layer,delete_channels
model_13 = delete_channels(model_12, model_12.layers[49],bb)
model_13.summary()
model_13.save('ent_s_cifar10vgg16_13.h5')
model_13.save_weights('ent_s_cifar10vgg16_13_weights.h5',overwrite=True)

Deleting 357/512 channels from layer: conv2d_13
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 35)        980       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 35)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 35)        140       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 35)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 18)        5688      
_________________________________________________________________
activation_2 (Activation)   

In [17]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model_13.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])
 
train_history=model_13.fit(x_train,y_train,epochs=200, batch_size=128,
             validation_split=0.1, verbose=1)

model_13.save('ent_s_cifar10vgg16_13.h5')
model_13.save_weights('ent_s_cifar10vgg16_13_weights.h5',overwrite=True)

Train on 45000 samples, validate on 5000 samples
Epoch 1/200
45000/45000 [==============================] - 24s 529us/step - loss: 0.7037 - acc: 0.9262 - val_loss: 1.0838 - val_acc: 0.8486
Epoch 2/200
45000/45000 [==============================] - 17s 373us/step - loss: 0.7260 - acc: 0.9232 - val_loss: 1.0330 - val_acc: 0.8550
Epoch 3/200
45000/45000 [==============================] - 18s 396us/step - loss: 0.7400 - acc: 0.9214 - val_loss: 1.0107 - val_acc: 0.8512
Epoch 4/200
45000/45000 [==============================] - 18s 400us/step - loss: 0.7433 - acc: 0.9227 - val_loss: 1.1009 - val_acc: 0.8302
Epoch 5/200
45000/45000 [==============================] - 18s 400us/step - loss: 0.7387 - acc: 0.9252 - val_loss: 1.1153 - val_acc: 0.8350
Epoch 6/200
45000/45000 [==============================] - 18s 393us/step - loss: 0.7488 - acc: 0.9229 - val_loss: 1.0570 - val_acc: 0.8512
Epoch 7/200
45000/45000 [==============================] - 18s 398us/step - loss: 0.7387 - acc: 0.9257 - val_lo

Epoch 59/200
45000/45000 [==============================] - 18s 400us/step - loss: 0.7223 - acc: 0.9302 - val_loss: 1.1208 - val_acc: 0.8352
Epoch 60/200
45000/45000 [==============================] - 18s 399us/step - loss: 0.7317 - acc: 0.9268 - val_loss: 1.0276 - val_acc: 0.8524
Epoch 61/200
45000/45000 [==============================] - 18s 404us/step - loss: 0.7196 - acc: 0.9319 - val_loss: 1.0539 - val_acc: 0.8550
Epoch 62/200
45000/45000 [==============================] - 18s 401us/step - loss: 0.7151 - acc: 0.9326 - val_loss: 1.0486 - val_acc: 0.8532
Epoch 63/200
45000/45000 [==============================] - 18s 397us/step - loss: 0.7185 - acc: 0.9322 - val_loss: 1.0857 - val_acc: 0.8426
Epoch 64/200
45000/45000 [==============================] - 18s 401us/step - loss: 0.7196 - acc: 0.9308 - val_loss: 0.9881 - val_acc: 0.8638
Epoch 65/200
45000/45000 [==============================] - 18s 401us/step - loss: 0.7209 - acc: 0.9315 - val_loss: 1.0786 - val_acc: 0.8464
Epoch 66/200


Epoch 117/200
45000/45000 [==============================] - 18s 398us/step - loss: 0.7119 - acc: 0.9328 - val_loss: 1.0520 - val_acc: 0.8488
Epoch 118/200
45000/45000 [==============================] - 18s 404us/step - loss: 0.7089 - acc: 0.9326 - val_loss: 1.0331 - val_acc: 0.8510
Epoch 119/200
45000/45000 [==============================] - 18s 404us/step - loss: 0.7168 - acc: 0.9284 - val_loss: 1.0379 - val_acc: 0.8508
Epoch 120/200
45000/45000 [==============================] - 18s 397us/step - loss: 0.7199 - acc: 0.9292 - val_loss: 1.0588 - val_acc: 0.8536
Epoch 121/200
45000/45000 [==============================] - 18s 404us/step - loss: 0.7121 - acc: 0.9303 - val_loss: 1.1067 - val_acc: 0.8360
Epoch 122/200
45000/45000 [==============================] - 18s 398us/step - loss: 0.7121 - acc: 0.9314 - val_loss: 1.0704 - val_acc: 0.8466
Epoch 123/200
45000/45000 [==============================] - 18s 400us/step - loss: 0.7107 - acc: 0.9308 - val_loss: 1.0076 - val_acc: 0.8566
Epoch 

Epoch 175/200
45000/45000 [==============================] - 20s 446us/step - loss: 0.7114 - acc: 0.9308 - val_loss: 1.0226 - val_acc: 0.8546
Epoch 176/200
45000/45000 [==============================] - 21s 457us/step - loss: 0.7102 - acc: 0.9301 - val_loss: 1.0166 - val_acc: 0.8598
Epoch 177/200
45000/45000 [==============================] - 21s 457us/step - loss: 0.7184 - acc: 0.9284 - val_loss: 1.0015 - val_acc: 0.8588
Epoch 178/200
45000/45000 [==============================] - 20s 454us/step - loss: 0.7100 - acc: 0.9326 - val_loss: 1.0263 - val_acc: 0.8542
Epoch 179/200
45000/45000 [==============================] - 21s 457us/step - loss: 0.7054 - acc: 0.9328 - val_loss: 1.0172 - val_acc: 0.8632
Epoch 180/200
45000/45000 [==============================] - 20s 452us/step - loss: 0.7121 - acc: 0.9303 - val_loss: 1.0356 - val_acc: 0.8546
Epoch 181/200
45000/45000 [==============================] - 21s 461us/step - loss: 0.7157 - acc: 0.9293 - val_loss: 1.0248 - val_acc: 0.8568
Epoch 

In [18]:
import functools
import keras
top1_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=1) #top-1精度

top1_acc.__name__ = 'top1_acc'

model_13.compile(loss='categorical_crossentropy',
             optimizer='adam',metrics=['accuracy','top_k_categorical_accuracy',top1_acc])

model_13.evaluate(x_test,y_test,verbose=0)

[1.0384635814666747, 0.8481, 0.9899, 0.8481]

In [19]:
score=model_13.evaluate(x_test,y_test,verbose=0)
print("测试损失为：{:.4f}".format(score[0]))
print("测试准确率为：{:.4f}".format(score[1]))

测试损失为：1.0385
测试准确率为：0.8481
